In [ ]:
from IPython.display import HTML


# Foreword

This is a developing notebook which will be updated regularly until this note is removed.

# Walkthrough on how to use the Notebook

In [ ]:
HTML("""
<iframe width="921" height="518" src="https://www.youtube.com/embed/E86oJMM1q30" title="Macro Economics Insight Notebook Walkthrough" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
""")

In [ ]:
!pip install arch
!pip install jupyter_dash
!pip install webcolors

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import pandas as pd
import pandas_datareader.data as web
from datetime import datetime, timedelta, date
import plotly.express as px
import plotly.graph_objects as go
from arch import arch_model
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from jupyter_dash import JupyterDash

from dash import Dash, dcc, html
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate
from webcolors import *
from IPython.display import display, Markdown
import warnings



from enum import Enum
import statsmodels.tsa.stattools as sta
from statsmodels.tools.sm_exceptions import ValueWarning, HessianInversionWarning, ConvergenceWarning

warnings.filterwarnings('ignore', category=ValueWarning)
warnings.filterwarnings('ignore', category=HessianInversionWarning)
warnings.filterwarnings('ignore', category=ConvergenceWarning)
 # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<a id=tbc></a>
<i> Unfortunately the cell links do not work in normal Kaggle view mode - to utilize the Table of contents,it is advised to try and actually open the notebook</i>
# Table of Contents

* [Overview & Objectives](#overview)
* [Commentary](#commentary)
* [What is happening with the US Population](#population)
* [What has been happening with CPI and Expecations?](#cpi_intro)
    - [Violin Distribution of CPI Expectations](#cpi_violin)
    - [CPI Expectations vs Spot Inflation](#cpi_spot)
    - [CPI Expectations vs Real Rates and Fed Policy Rates](#cpi_rates)
* [What has been happening to GDP Growth?](#gdp_intro)
    - [GDP Autocorrelation Plots](#gdp_ts_acf)
    - [GDP Time Series Forecasts](#gdp_ts_forecast)
* [Money Supply Trends](#money_supply)
* [What has been happening in Equities?](#equities)
* [What has been happening in FX](#fx)
   
    

<a id=overview></a>
# Overview 

Current Economic conditions appear dire but the hypothesis is that they can be attributed to a few root causes. The purpose of this notebook will be to examine the drivers to key economic variables (i.e. inflation, growth and income) and the impact of past policy rates impacts on inflation and unemployment.

# Objectives
    - Understand recent macro economic events relative to big picture (as catch up since I've been out of the markets myself)
    - forecast where we expect key macro econmic variables:
        - Inflation Rate 
            - 1 , 3, 6, 1y forward
        - GDP
            - Next, 3m, 6m, 1y, 2y
        - FX
            - 1w, 1m, 3m, 6m, 1y
        - Rates
            - 1w, 1m, 3m, 6m, 1y
        - Equities
            - 1w, 1m, 3m 6m, 1y
            



<a id="commentary"></a>


# Week of Oct 31 (Happy? Halloween) Commentary

### Big Global Themes

##### GeoPolitical Heavies 
- **UK**  - Rishi Sunak formalizes his position as PM though as expected to a slightly rocky start. [UK property prices begin to fall as higher interest rates bite into demand for homes](https://www.ft.com/content/7f2d77f2-4221-4b99-b939-373a70b1330c). Not to mention, the after effects of the Gilts volatility now [continues to affect UK companies](https://www.ft.com/content/d8f503c8-ceb1-4f13-80da-7e596bf88e9d). Given the event risk of the UK PM transition has just occurred, I think a more appropriate report card time will be in 2 weeks where there will be more concrete evidence (hopefully) on the strategy and implemenetation by the new governement.
- **US**
    - In politics, [Republicans are leading slightly and looking to win in previously Red Swing states](https://www.nytimes.com/2022/11/01/us/politics/midterm-elections-governor-key-races.html). 
    - In economics, The major print so far this week has been PMIs and ISMs which still point to expansion though the attention of the week will be on Fed Rates on Wednesday and probably NFP on Friday. [JPM has a call for 75bps (though is also hinting that dovish language should start emerging and that hikes should slow to 50 then 25bps in Q123)](https://podcasts.apple.com/us/podcast/at-any-rate/id1481423053?i=1000584275348), [MS seems to think there are possibilities for a hold since Money Supply has experienced its first contraction in a long time and there seems to be signs that prices are cooling and have what they call is a "tactical" long on US Equities](https://podcasts.apple.com/us/podcast/thoughts-on-the-market/id1466686717?i=1000584566077). [Somehow today's PMI prints and articles like these suggest that there is still enough demand in the system to sustain higher prices as restaurants / groceries somehow maintain margins](https://www.nytimes.com/2022/11/01/business/food-prices-profits.html). I'm expecting there will need to be some sort of retracement of demand in another market if Friday's NFP will probably show Average Hourly Earnings trail CPI - and numbers come below expectations - NFP consensus at 200k ([JPM call is for 175k NFP on Friday](https://podcasts.apple.com/us/podcast/global-data-pod/id1576857391?i=1000584290942)). My personal call is that Fed Hikes at market consensus of 50bps or JPM's call for 75bps - [the rates curve is showing real rates are still pretty low](#cpi_rates) so I don't see the Fed "tactically" pausing when Real GDP is still comfortably at 2.6% despite high inflation, PMIs/ISM still hints a timid expansion (though elements are weaker) - especially when elements of food inflation and the war persists.
- **China** - Pretty sure everyone's seen the video of Hu Jintao's dismissal from the Chinese Congress and Western Media's various speculations as to the reasions - [more important appears to be the change up in his cabinet and the 7 new people of power](https://www.ft.com/content/758019bb-418d-4c89-95f6-0c4dfea70281). Long story short, Xi has dispensed with traditional norms and elected members that seem to have been loyalists for many years. Xi's policies seem more centered on reclaiming Taiwan and the "Senkaku" Islands that are in dispute with Japan. Given other global tensions, China's restructuring will be important though I'd be surprised to see any major tensions ahead of the Chinese New Year (which would be Xi's last big chance to shore up some revenue for the reserves).
- **Ukraine** - [Grain deal collapse is the latest concern to elevate fears on global food inflation](https://www.ft.com/content/544fd511-eacb-48ee-aff1-b53b112f77de) despite almost all banks hiking currently and expected to continue into the near future. [The live map from the FT for free](https://www.ft.com/content/4351d5b0-0888-4b47-9368-6bc4dfbccbf5) suggests major incursions in 3 of the 4 regions recently annexed by Russia - let's see how we do going into the winter.
- **Europe** - [Eurozone inflation already hits 10.7% despite the recent 25bp hike](https://www.ft.com/content/d783e38e-7a58-4285-b68a-55e357bb8c4b) and is now 5x the ECB's target 2%.

##### Big Picture Concerns from Smaller Markets
- General Emerging Markets
    * **[Brazil](https://www.ft.com/content/00201127-baec-47e7-877e-60c18c64fbe5)** - Lula is now in in power - allegedly. Bolsonaro's promise of reprise if he lost is still waiting to mateirialize though hopefully the dog barks loudly and suffers tooth decay. I think the big question now is going to be [economic vision from the new Presidency](https://www.ft.com/content/34fc10e0-0519-4dcb-9e31-16477ebdfd39) - most foreign investors have been wary of a Lula return due to previous lax fiscal policies (and an already vulnerable fiscal backdrop) though [yields are generally much lower post election](https://www.ft.com/content/6daad790-165a-498a-8287-8a41a1a265fd). Though until Bolsonaro concedes (which he hasn't I believe).
    * **[Egypt](https://www.ft.com/content/03533d92-4a71-43fc-b885-27dcb962d4e8)** - Sisi's promise of economic recovery after he took power has long broken as well as his promises on human rights and other violations. Now with the economy sharply sinking and the Currnet Account spiralling out of control, Sisi is embarking the similar path of most dictator's before him and peddling for an IMF handouts or hoping for large deposits from Saudi to the central bank to stabilize the economy.
    * **[India](https://www.ft.com/content/5464aff7-61da-40bb-9b66-e3f45eda32a1)** - Large investments are being made into India's 5G network as India's 2 largest billionares - Ambanai and Adani - seek to grow 5G in the consumer and enterprise markets respectively. While I am keen to see 5G adoption rates, a larger 5G market presents a huge opportunity to phone manufacterers and service providers though American based providers will likely struggle with the stronger dollar to maintain profitability. Also, with [reminders of weak infrastructure in broad parts of India](https://www.ft.com/content/8abc2fde-2491-4eb4-8aa0-a912d9df9eb9), am curious to see how they will implement the complex architecture associted with 5G.
- Climate
    * **Europe** - [France](https://www.ft.com/content/48178f1e-5572-496e-8fbe-bb4c0e3ec8ea) - after withdrawl from the Energy Charter Treaty last week, France/Brussels now demands fair treatment to Euro-Block clean vehicles which are not entitled to same subsidies as their American counterparts. [Germany](https://www.ft.com/content/c7ea1893-75a4-4fa2-b524-aa5df625ee91) steps up financing of Climate initatives and seaks to overhaul World Bank (especially post recent IMF/ WB climate protests).
    * **[Oil](https://www.ft.com/content/1fd06f38-ec60-4043-bcdd-adcba8beb006)** - IEA estimates that fossil fuel demand will finally peak this decade and if COUNTRIES WOULD JUST WORK TOGETHER we could get Non-fossil fuel consumption/supply to exceed Fossil Fuels by as early as 2035.
    
    
    
### Markets Views
<span style="color:red">*Disclaimer*</span>:
<i>Not being in the markets, I wholely admit that I don't have the access to data that would make my commentary here a lot more useful, but I'll do the best I can with what we got. As always, I always recommend questioning away until you don't have any doubts (and try to find as many doubts as possible) because when it comes to markets, there is usually more opinion than fact. Please try to inform yourselves to the best of your ability and treat the notes in this document as views to give perspective to your own.</i>

*Post Fed Minutes Update*<br>
Post the Fed minutes the markets should now be pricing in a series of longer hikes. JPM says end of Q2 which doesn't sound unrealistic -  From Powell's language, there seems to be no immediate need for stopping fed hikes as O/N real rates appear to be negative (though 

* **Equities** - Risk markets have generally performed well last week though as expected they are retracing after the GDP print. I am coming off my 4.1k call on S&P (despite MS's call for 4150 T/P call) -  I think there is a chance for tactical move higher though given recent combination of strong prints my inclination is that we are going to have another round of hawkish-ish language I am expecting for equity markets to stall at the end of this week and retrace lower to 3.7k in S&P and Nasdaq 10.6k. I think after touching the lows we can then retrace back higher to 4.1k (as we comfortably clear November Monetary event risk and move into last weeks of Nov / first weeks of Dec).
    - Meta crash and vol is the going theme. Last weeks' earnings were concerning more due to the raise in CAPEX rather than a reduction in earnings - though that wasn't nice either. Hard to say how there is room for the stock to return for another quarter until there is some major headline with respect to management / strategy change that the market will positively absorb. Right now stocks seem at okay value since P/E is now lower than most tech stocks - but if they keep burning through cash is it low enough? Use anything close to 100 to sell your shares in my opinion.
    - Tesla shares are struggling to break 230 despite Elon selling pressure now clearly out of the market. I guess concerns are raised over the fact that Twitter and Tesla fates are becoming closely entwined. I personally still expect these stocks to lag the S&P so would sell if you can get to 230 or close to it. I expect them lower than 200 before end of the year.
    - Financials & Energy are probably top contenders for my picks still. JPM shares look cheap in my opinion (there is some BS about PB and WM grudge internally, all I know is that they have huge balance sheet, lots of deposits and great macro desk - will probalby have good year). GS will also probably do well, has less market advantage in my opinion than JPM. European energy probably better choice (BP, Shell) as these companies will be needed now even if they had PR blips in the past. Invest in BP if possible!. American Energy has had blockbuster year, will probably continue.



* **Rates** -  [JP Morgan is calling for a 75bp hike in Nov follwed by 50bps in Dec to the Fed Policy Rate](https://podcasts.apple.com/us/podcast/at-any-rate/id1481423053?i=1000583872389) - Given 1y at 4.60-4.70, 2y Treasuries are around 4.40-4.50 and 10y at 4.0 - 4.10 (down ~5bps) right now and has held for a week - though at the high end of these ranges - I expect we are 10bps higher in all front rates in 1 week. In 2006 we hit a high of ~5.25% in fed policy rates which translated to just below 5.2% in 2y and ~6.25% in 1y Treasury yields. JPM thinks yearend 10y rates are going to be 4.2 - 4.5/6 (upside).I agree with most Banks calls for signs of deceleration in hiking for future cycles but I think we need to see evidence that real rates are closer to +2% before the fed starts decelerating to 50bps. My guess is that real rates need to print at around +1-1.25% or signs of an impending strong Q4 contraction for Fed to start easing the gas here. 

* **FX** - USD is coming off the 3y highs as other currencies finally rally (namely EUR and GBP) though I expect that to end this week (and growth in other markets as a result). One last hawkish Fed should probably clear out any recent dollar shorts. This is EUR will probably continue to appreciate as demand for energy will probably be more severe there and put more cash demand. GBP will probably enjoy revival another couple weeks. I still like the long EURBRL trade - so far foriegn investors have been favorable to Lula but that just means to me that there should be more FX vol premium (because if foreigners get spoked they will exit positions to hard currency). With eur inflation so high,  I expected more hawkish policy to keep demand for EUR strong. I'm timidly thinking my em local index should start doing better here as I would imagine after a short term shake up demand for dollar needs to cool off or the Fed will face a new problem.
* **Credit** - No idea, wish I was in the bond market. Gotta find more info.
* **Real Estate** - We've seen light buying in our market but in general deal flow has collapsed in SF bay area.





<details><summary><span style="color:maroon">Click Here for  Week of Oct 24 Commentary</span></summary>
    
# Week of Oct 24 Commentary

### Big Global Themes

##### GeoPolitical Heavies 
- **UK**  - Rishi Sunak becomes UK PM - Gilts rally, Blind blows up in celebrations and snarky debate. Now the big question is emerging on next cabinet and whether Sunak can balance a tricky need for growth and a demonstration of fiscal responsibility.
- **US** - This weeks' big question is what happened to GDP this week comes up 2.6% (inline with y AR1 model but generally above my expectations)~~(median 2.3% forecast, Fed Atlanta 3.1% coming Thursday at 8:30 am EST)~~ and what could that mean for employment and inflation ahead of the fed's next meeting in Nov? Time Series projections using a moving average suggest a much lower 3Q and higher 4Q Real GDP Given July '20 was a very high print and recent Q1 and Q2 have been very weak. Testing regression techniques to see what signals emerge.
- **China** - Xi back in it for round 3 which has [mild revulsion in the China markets](https://www.ft.com/content/1dff4b82-151d-4689-900a-5d968837faa8) (and likely felt in other respects in other corners of the world). The more specifc catalyst appears to be Softbank's sell off of Alibaba steak but there seems to be general unease about the reshuffle in the cabinet.
- **Ukraine** - Russia: Ukraine's fight against Putin continues in the South and East and appearing to be leaning in the favor of the Ukrainians on most western publications currently. The big question is nuclear - there were concerns that [Shoigu could use false "dirty bomb" threasts](https://www.ft.com/content/792fc80c-e3a5-4ce3-8144-db2ff29132b7) as a pretence for a nuclear strike but that threat seems to have subsided
- **Europe** - [Commodities slip below 100 eur for the first time since the blow up with Russia](https://www.ft.com/content/8d2d1e54-b2b7-4213-a1c4-daf9d42884cc) and apparently European Gas prices are down 70% since August. 

##### Big Picture Concerns from Smaller Markets
- General Emerging Markets
    * **[Brazil](https://www.ft.com/content/542e5046-15f5-4ec0-b598-068cc03622da)** - October 30th is the big show down betwen Lula and Bolsanaro and there are already big disputes breaking out over the election voting mechanism and positioning Brazil for a shaky start to November. Whoever was smart enough be long eur/brl vol is probably rolling in it right now.
    * **[South Africa](https://www.ft.com/content/f2389925-a9e1-4402-9056-e4ef137ebae7)** - Ramaphosa's most recent establishment of a Anti-Graft anti-corruption agency (long awaited post the Zuma / Gupta Saga and corresponding country wide rioting and unrest). This hopefully will offset concerns of [State Failure](https://www.ft.com/content/79123d0a-af62-43e3-bfbe-ae942224cea7) and the broader associated concerns with the region.
    * **[Saudi Arabia](https://www.ft.com/content/92ae04c5-3024-4fc1-8862-0a0463f34dab)** - Banking heads convering on Saudia Arabia continues to fuel its position as a local Geopolitical power in the Middle East. Tensions to look out for will be the reprecussions for recent OPEC cuts / unfavorable US pricing and general operating in a framework at odds with most Western Principles and more inline with China / Russian governments.
- Climate
    * **Europe** - [France](https://www.ft.com/content/4d33e509-176a-4f44-9846-74073d7056ab) - withdrawl from the Energy Charter Treaty probably a big blow atleast in terms of messaging / symbolism for the country that was the proponent behind the Climate Paris Accords. On the other hands [Germany](https://www.ft.com/content/c7ea1893-75a4-4fa2-b524-aa5df625ee91) steps up financing of Climate initatives and seaks to overhaul World Bank (especially post recent IMF/ WB climate protests).
    * **[Brazil](https://www.ft.com/content/ef25c015-bf13-40c3-9013-261c5f7bb103)** - As fears of Bolsonaro's pro-destroy-the-amazon policies seem at risk of coming to an end, loggers in the Amazon are in a frantic rush to nail the coffin on one of the most important ecological preserves in the world.
    
    
    
### Markets Views
<span style="color:red">*Disclaimer*</span>:
<i>Not being in the markets, I wholely admit that I don't have the access to data that would make my commentary here a lot more useful, but I'll do the best I can with what we got. As always, I always recommend questioning away until you don't have any doubts (and try to find as many doubts as possible) because when it comes to markets, there is usually more opinion than fact. Please try to inform yourselves to the best of your ability and treat the notes in this document as views to give perspective to your own.</i>

*Thursday Update*<br>
After GDP print in my opinion the S&P will have less strength to accelerate given there has been little material slow down in the real economy (atleast according to the metrics viewed by the Fed) so there is probably more room for hikes in the future and less room for equities growth. I think less chance we see S&P hit 4k next week as I expect stocks to slow significantly in the lead up to Fed Nov 12th meeting (also near CPI print).

* **Equities** - Risk markets have generally performed well from the start of the week. After mounting a timid recovery at the end of last week, the S&P, Nasdaq and Dow have all benefited generous rebounds from the lows and are reaching the near term highs ahead of Thursday's GDP print (if we see [very] low GDP, I think there is a chance of a small rally to S&P 4-4.1k, Dow 32.3k, Nasdaq 12.2k (though Nasdaq has had weakest performance and something I haven't yet investigated).
    - Tesla, NVIDIA had temporary revivals on Tuesday after poor earnings and underperforming the S&P. My guess is that these will probably have reached their highs on Tuesday until Thursday's GDP print. If weak GDP, we may see the junky tech stocks with high PE still rally (TDD) - recommend dumping on the highs (after seeing where levels stabilize ~2hrs after market open). Until we see major econmic collapse I imagine we see these companies under perform the market.
    - Financials & Energy are probably top contenders for my picks still. JPM shares look cheap in my opinion (there is some BS about PB and WM grudge internally, all I know is that they have huge balance sheet, lots of deposits and great macro desk - will probalby have good year). GS will also probably do well, has less market advantage in my opinion than JPM. European energy probably better choice (BP, Shell) as these companies will be needed now even if they had PR blips in the past. Invest in BP if possible!. American Energy has had blockbuster year, will probably continue.



* **Rates** -  [JP Morgan is calling for a 75bp hike in Nov follwed by 50bps in Dec to the Fed Policy Rate](https://podcasts.apple.com/us/podcast/at-any-rate/id1481423053?i=1000583872389) - seems reasonable to me unless there is a huge upset with GDP and sudden evidence of job market weakness.Hard to say without seeing what's trading directly, but if I had to guess given 1y at 4.60-4.70, 2y Treasuries are around 4.40-4.50 and 10y at 4.10 - 4.20 right now, unless we see a very low GDP (i.e. negative) it seems difficult for rates to stay below 4.5% if GDP and inflation remain strong. In 2006 we hit a high of ~5.25% in fed policy rates which translated to just below 5.2% in 2y and ~6.25% in 1y Treasury yields. JPM thinks yearend 10y rates are going to be 4.2 - 4.5/6 (upside). While I hunt for the data, I am guessing that Fed Dots also forecasted ~6.5% policy rates in the upcoming year only to be surprised by a much nastier than expected recession in 2008. Back in 2005, the economy also had an inverted inflation expectation curve (pretty similar to now) - given this data my guess is that 
* **FX** - USD is coming off the 3y highs as other currencies finally rally (namely EUR and GBP). This is EUR will probably continue to appreciate as demand for energy will probably be more severe there and put more cash demand.GBP will probably get a "glam up" revive with Sunak returning for next few weeks.
* **Credit** - No idea, wish I was in the bond market. Gotta find more info.
* **Real Estate** - Property appears to be dropping like it's hot (especially in the SF Bay Area) - I expect this will need to continue for another 6 months atleast (finger in the air guess here from all the continues Tech layoff headlines and high interest rates). Expecting worst markets in SF and Seattle. After a recent visit to a red state, I have a funny feeling that red states are going to enjoy relatively better returns than blue states (eerily no homless, much cleaner streets, percieved cheaper living) which will probably attract migrants from blue states pushed by recession. Charts on this coming soon.

</details>


<a id=population></a>
# US Population
    - Household estimates - TTLHHM156N
    - Total US population - POPTHM

[Return to Table of Contents](#tbc)

In [ ]:
# Pulling Household and Population data
house_hold_estimates = web.DataReader("TTLHHM156N", 'fred', datetime(1980, 6, 1), datetime.now())
house_hold_estimates.columns = ['household_estimates']

total_us_population = web.DataReader("POPTHM", 'fred', datetime(1980, 6, 1), datetime.now())
total_us_population.columns = ['total_us_population']


In [ ]:
# Visualizing Population Data
population_df = pd.concat([house_hold_estimates, total_us_population], axis=1)

population_df['Average_population_per_household'] = population_df['total_us_population']/population_df['household_estimates']
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.update_layout(title={'text': "Total US Population vs Household"}, yaxis={'title': "Population"}, yaxis2={'title': "Avg Population per Household"}, width=1000)
fig.add_trace(go.Scatter(x=population_df.index, y=population_df[population_df.columns[0]], name=population_df.columns[0], mode='lines'))
fig.add_trace(go.Scatter(x=population_df.index, y=population_df[population_df.columns[1]], name=population_df.columns[1], mode='lines'))
fig.add_trace(go.Scatter(x=population_df.index, y=population_df[population_df.columns[2]], name=population_df.columns[2], mode='lines'), secondary_y=True)
fig.show()

In [ ]:
# Average Monthly percent change 
(pd.concat([house_hold_estimates, total_us_population], axis=1).pct_change().dropna() * 100).mean()

## Observations on Population

An overview of Population trends shows that US population has regularly grown by ~7%/yr while household estimates has grown by avewrage 9.5%/yr resulting in number of population per household compressing to closer to 2 from 3 (perhaps as a signal of lower fertility / birth rates?)

In summary though there are currently a total of ~333 mio people in the US and ~128 mio people

<a id=cpi_intro></a>
# CPI

* Data from Fred
    - Sticky CPI less energy and food - CORESTICKM158SFRBATL
    - Median CPI - MEDCPIM158SFRBCLE
    - Expected Inflation?
        - 1y - EXPINF1YR
        - 2y - EXPINF2YR
        - 3y - EXPINF3YR
        - 4y - EXPINF4YR
        - 5y - EXPINF5YR
        - 7y - EXPINF7YR
        - 10y - EXPINF10YR
        - 20y - EXPINF20YR


* Question being considered
    - using TSA where can we project Expected Inflations to go from here?
    - Can we improve on TSA using regression methods (start with linear, then tree methods, )
    


[Return Home](#tbc)

In [ ]:
inflation_years = [1, 2, 3, 4, 5, 7, 10, 20]

inflation_curves = web.DataReader([f"EXPINF{i}YR" for i in inflation_years], 'fred', datetime(1970, 6, 1), datetime.now())

<a id=cpi_violin></a>

<i> The following chart aims show a distribution of inflation expectations across different presidential terms.
Please click the different presidential terms to compare against different terms.</i>

[Return Home](#tbc)

In [ ]:

# Violin plot of the Inflation Expecations

legend_groups = {
    "Reagan": (datetime(1981, 1, 20), datetime(1989, 1, 20)),
    "H-Bush": (datetime(1989, 1, 20), datetime(1993, 1, 20)),
    "Clinton": (datetime(1993, 1, 20), datetime(2001, 1, 20)),
    "W-Bush": (datetime(2001, 1, 20), datetime(2009, 1, 20)),
    "Obama": (datetime(2009, 1, 20), datetime(2017, 1, 20)),
    "Trump": (datetime(2017, 1, 20), datetime(2021, 1, 20)),
    "Biden": (datetime(2021, 1, 20), datetime.now())
}

color_outlines = {
    "Reagan": "wheat",
    "H-Bush": "magenta",
    "Clinton": "turquoise",
    "W-Bush": "red",
    "Obama": "slateblue",
    "Trump": "thistle",
    "Biden": "yellowgreen"
}

color_groups = {
    "Reagan": "tomato",
    "H-Bush": "tomato",
    "Clinton": "slateblue",
    "W-Bush": "tomato",
    "Obama": "slateblue",
    "Trump": "tomato",
    "Biden": "slateblue"
}


fig = go.Figure()
fig.add_trace(go.Scatter(x = [f"{i} YR" for i in inflation_years], y=inflation_curves.tail(1).values[0], name="Most Recent Inflation", line={'color': "blue"}))
for key, dts in legend_groups.items():
    inflation_curve_subset = inflation_curves.loc[(inflation_curves.index >= dts[0] ) & (inflation_curves.index < dts[1])]

    inflation_curve_subset.columns = [f"{i} YR" for i in inflation_years]
    for col in inflation_curve_subset.columns:
        fig.add_trace(go.Violin(y=inflation_curve_subset[col].values, 
        x=[col]*inflation_curve_subset[col].shape[0], 
        name=col,
        legendgroup=f"{key} Years", 
        legendgrouptitle_text = f"{key} {dts[0].strftime('%Y')} - {dts[1].strftime('%Y')} Years" , 
        fillcolor=color_groups[key], 
        line_color=color_outlines[key]))


layout = go.Layout(
    title = "Violin Distribution of Inflation Expectations<br>Split by Presidential Terms",
    xaxis = {'title': "Inflation Expectation Term"},
    yaxis = {'title': "Inflation Rate"},
    height=800
)

fig.update_layout(layout)

## Observations on historical inflation expectations
Reviewing the different terms, we are still a long stretch from the high inflation expecations we saw during the Reagan era but we have materially higher near term (1-3y) inflation expecations (and the widest range since the 80s). For longer-term inflation expecations, the Biden era has actually had typically lower longer term inflation expecations suggesting that the Fed's committment to tackle inflation is being taken moderately seriously



In [ ]:
spot_inflation = web.DataReader(["CORESTICKM158SFRBATL", "MEDCPIM158SFRBCLE", "DPCCRV1Q225SBEA"], 'fred', datetime(1970, 6, 1), datetime.now())

<a id=cpi_spot><a/>

# Inflation Expectations vs Current Readings
[Return Home](#tbc)

In [ ]:
# Making Future Inflation Expectations Curve Figure
frames = []

sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Year:",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}


fig = make_subplots(rows=2,cols= 1, subplot_titles = ("Inflation Expectations", "Spot Inflation Prints"))
fig.update_layout(height = 650, paper_bgcolor='white', plot_bgcolor = "white", updatemenus=[dict(type="buttons", buttons=[   {
                "args": [None, {"frame": {"duration": 500, "redraw": False},
                                "fromcurrent": True, "transition": {"duration": 300,
                                                                    "easing": "quadratic-in-out"}}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        ])],xaxis={'title': "Number of Years", 'gridcolor': "lightgrey"} , 
        xaxis2={
            'range': [spot_inflation.index.min()- pd.DateOffset(months=6), spot_inflation.index.min()+ pd.DateOffset(months=6)],
            'title': {'text': 'Date'}
        }, 
        yaxis= {
            'range':[-1, 8],
            'title': {'text': 'Inflation Rate (%)'},
            'zeroline': True,
            'gridcolor': 'lightgrey',
            'zerolinecolor': 'black'
            },
        yaxis2 = {'title': "Inflation Rate (%)" , 'zeroline': True, 'zerolinecolor': 'black', 'gridcolor': 'lightgrey'},
        title={'text': "Future Inflation Expectations vs Spot Inflation Metrics"},
        
        barmode='group')

# fig = go.Figure(data, layout, frames)
for i, ix in enumerate(inflation_curves.index):
    spot_subset = spot_inflation.loc[spot_inflation.index <=ix]
    fig.update_layout(xaxis2={'range': [ix - pd.DateOffset(months=10),ix + pd.DateOffset(months=2)]})
    newLayout = fig.layout
    # newLayout.yaxis['range'] = [inflation_curves.loc[ix, :].min()-1.5, inflation_curves.loc[ix, :].max() + 1.5]
    if i == 0:
        fig.add_trace(go.Scatter(x=inflation_years, y=inflation_curves.loc[ix, :].values, name="Current Inflation Expectations", legendgroup="Inflation Expectations", line={'color': '#9CC6DE'}),row= 1, col= 1)
        fig.add_trace(go.Bar(x=spot_subset.index, y=spot_subset["CORESTICKM158SFRBATL"].values, name="Sticky Price inflation", legendgroup = "Spot Inflation", xaxis='x2'), row=2, col=1)
        fig.add_trace(go.Bar(x=spot_subset.index, y=spot_subset["MEDCPIM158SFRBCLE"].values, name="Core Median inflation", legendgroup="Spot Inflation", xaxis='x2'), row=2, col=1)
        fig.add_trace(go.Bar(x=spot_subset.index, y=spot_subset["DPCCRV1Q225SBEA"].values, name="Core PCE inflation", legendgroup="Spot Inflation", xaxis='x2'), row=2, col=1)
        fig.add_trace(go.Scatter(line={'dash': 'dash', 'color': '#3116F5'}, name="3m prior", legendgroup = "Inflation Expectations"),row= 1, col= 1)
        fig.add_trace(go.Scatter(line={'dash': 'dot', 'color': '#1C0C91'}, name="6m prior", legendgroup = "Inflation Expectations"),row= 1, col= 1)
    elif i>= 3 and i < 6:
        frames.append(go.Frame(data = [
            go.Scatter(x=inflation_years, y=inflation_curves.loc[ix, :].values, name="Current Inflation Expectations"), 
            go.Bar(x=spot_subset.index, y=spot_subset["CORESTICKM158SFRBATL"].values, name="Sticky Price inflation", xaxis='x2'), 
            go.Bar(x=spot_subset.index, y=spot_subset["MEDCPIM158SFRBCLE"].values, name="Core Median inflation", xaxis='x2'),
            go.Bar(x=spot_subset.index, y=spot_subset["DPCCRV1Q225SBEA"].values, name="Core PCE inflation", xaxis='x2'),
            go.Scatter(x=inflation_years, y=inflation_curves.loc[ix-pd.DateOffset(months=3), :].values, name="3m prior")],
            layout=newLayout, name=ix.strftime('%d-%b-%Y')))
        # frames.append(go.Frame(data = go.Scatter(x=inflation_years, y=inflation_curves.loc[ix-pd.DateOffset(months=3), :].values, name="3 months prior"), name=ix.strftime('%d-%b-%Y')))
        # frames.append(go.Scatter(x=inflation_years, y=inflation_curves.loc[ix, :].values, name="3 Months Lag"))
    elif i >= 6:
        frames.append(go.Frame(data = [
            go.Scatter(x=inflation_years, y=inflation_curves.loc[ix, :].values, name="Current Inflation Expectations"), 
            go.Bar(x=spot_subset.index, y=spot_subset["CORESTICKM158SFRBATL"].values, name="Sticky Price inflation", xaxis='x2'), 
            go.Bar(x=spot_subset.index, y=spot_subset["MEDCPIM158SFRBCLE"].values, name="Core Median inflation", xaxis='x2'),
            go.Bar(x=spot_subset.index, y=spot_subset["DPCCRV1Q225SBEA"].values, name="Core PCE inflation", xaxis='x2'),
            go.Scatter(x=inflation_years, y=inflation_curves.loc[ix-pd.DateOffset(months=3), :].values, name="3m prior"),
            go.Scatter(x=inflation_years, y=inflation_curves.loc[ix-pd.DateOffset(months=6), :].values, name="6m prior")
            ],
            layout=newLayout, name=ix.strftime('%d-%b-%Y')))
    else:
        frames.append(go.Frame(data = [
            go.Scatter(x=inflation_years, y=inflation_curves.loc[ix, :].values, name="Current Inflation Expectations"), 
            go.Bar(x=spot_subset.index, y=spot_subset["CORESTICKM158SFRBATL"].values, name="Sticky Price inflation", xaxis='x2'), 
            go.Bar(x=spot_subset.index, y=spot_subset["MEDCPIM158SFRBCLE"].values, name="Core Median inflation", xaxis='x2'),
            go.Bar(x=spot_subset.index, y=spot_subset["DPCCRV1Q225SBEA"].values, name="Core PCE inflation", xaxis='x2')
        ],
            layout=newLayout, name=ix.strftime('%d-%b-%Y')))
        # frames.append(go.Frame(data = go.Bar(x=spot_subset.index, y=spot_subset["CORESTICKM158SFRBATL"].values, name="Core Median inflation", xaxis='x2'), name=ix.strftime('%d-%b-%Y')))
    slider_step = {"args": [
        [ix.strftime('%d-%b-%Y')],
        {"frame": {"duration": 100, "redraw": True},
         "mode": "immediate",
         "transition": {"duration": 100}}
    ],
        "label": ix.strftime('%d-%b-%Y'),
        "method": "animate"}
    sliders_dict["steps"].append(slider_step)
fig.update_layout(sliders = [sliders_dict])

# fig.layout = layout
fig.frames = frames
# fig = go.Figure(data, layout, frames)
fig

## Hint

 - user the slider to speed ahead and isolate on periods of interest

## Observations

The inflation expectation curve has inverted materially for the first time in ~40 years suggesting that the concerns concentrated in the near term but longer term inflation expecations are materially inching higher.

Even during the Reagan - Clinton error, when inflation rates were in general much higher, the curve remained relatively flat / upper sloping and the shape has persisted until only very recently.

# Inflation Expectations and Real Rates
[Return Home](#tbc)

In [ ]:
real_rates = web.DataReader(["REAINTRATREARAT1MO", "REAINTRATREARAT1YE", "REAINTRATREARAT10Y"], 'fred', datetime(1970, 6, 1), datetime.now())
real_rate_x = [1/12, 1, 10]

treasury_bonds_x = [1, 2, 5, 10, 20, 30]
treasury_bills_x = [1/12, 3/12, 6/12, 1]

us_policy_rates = web.DataReader(["DFF", "EFFR", "SNDR"], 'fred', datetime(1970, 6, 1), datetime.now())
treasury_bills = web.DataReader(["DTB4WK", "DTB3", "DTB6", "DTB1YR"], 'fred', datetime(1970, 6, 1), datetime.now())
treasury_bonds = web.DataReader([f"DGS{i}" for i in treasury_bonds_x], 'fred', datetime(1970, 6, 1), datetime.now())


newCurves = pd.concat([treasury_bills, treasury_bonds, real_rates], axis=1).ffill()

treasury_bills = newCurves[treasury_bills.columns]
treasury_bonds = newCurves[treasury_bonds.columns]

us_policy_rates.dropna(how='all', inplace=True)
us_policy_rates.ffill(inplace=True)

In [ ]:
implied_real_rates = pd.DataFrame()
for x in treasury_bonds_x[:-1]: 
    df = pd.concat([treasury_bonds[f"DGS{x}"], inflation_curves[f"EXPINF{x}YR"]], axis=1).dropna()
    implied_real_rates[f"Implied_{x}YR"] = df[f"DGS{x}"] - df[f"EXPINF{x}YR"]

<a id=cpi_rates></a>

[Return Home](#tbc)

In [ ]:
frames = []

sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Year:",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}


fig = make_subplots(rows=3,cols= 1, shared_xaxes =True, subplot_titles = ("Inflation Expectations", "Real Rates", "Treasury Bills & Bonds Curve"))
fig.update_layout(height = 650, paper_bgcolor='white', plot_bgcolor = "white", updatemenus=[dict(type="buttons", buttons=[   {
                "args": [None, {"frame": {"duration": 500, "redraw": False},
                                "fromcurrent": True, "transition": {"duration": 300,
                                                                    "easing": "quadratic-in-out"}}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        ])], yaxis={'range':[-1, 8], 'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"}, yaxis2={'range':[-8, 12], 'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': 'black'}, 
        yaxis3={'range': [0, 15], 'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"}, title={'text': "Future Inflation Expectations vs Real Rates & Nominal Rates"},
        barmode='group', bargap=.1)

for i, ix in enumerate(inflation_curves.index):
    newLayout = fig.layout
#     fig.update_layout(xaxis3={'range': [ix - pd.DateOffset(months=3),ix]})
    # newLayout.yaxis['range'] = [inflation_curves.loc[ix, :].min()-1.5, inflation_curves.loc[ix, :].max() + 1.5]
    if i == 0:
        fig.add_trace(go.Scatter(x=inflation_years, y=inflation_curves.loc[ix, :].values, name="Current Inflation Expectations", line={'color': '#9CC6DE'}, legendgroup="Inflation Exepctations"),row= 1, col= 1)
        fig.add_trace(go.Scatter(x=real_rate_x, y=real_rates.loc[ix, :].values, name="Real Rates Curve", xaxis='x2', line={'color': "#DEB6DA"}, legendgroup="Real Rates"), row=2, col=1)
        fig.add_trace(go.Scatter(x=treasury_bonds_x[:-1], y=implied_real_rates.loc[ix, :].values, name="Implied Real Rates Curve", xaxis='x2', line={'color': "#DEC2ED"}, legendgroup="Real Rates"), row=2, col=1)
        fig.add_trace(go.Scatter(x=treasury_bills_x, y=treasury_bills.loc[ix, :].values, name="Treasury Bill Curve",  line={'color': "#E5574C"}, legendgroup = "ST Nominal Rates"), row=3, col=1)
        fig.add_trace(go.Scatter(x=treasury_bonds_x, y=treasury_bonds.loc[ix, :].values, name="Treasury Bond Curve", line={'color': "#E57019"}, legendgroup = "LT Nominal Rates"), row=3, col=1)
        fig.add_trace(go.Scatter(line={'dash': 'dash', 'color': '#3116F5'}, name="3m prior", legendgroup="Inflation Exepctations"),row= 1, col= 1)
        fig.add_trace(go.Scatter(line={'dash': 'dash', 'color': "#DE8ED6"}, name="3m prior", legendgroup="Real Rates"),row= 2, col= 1)
        fig.add_trace(go.Scatter(line={'dash': 'dash', 'color': "#8D3730"}, name="3m prior", legendgroup = "ST Nominal Rates"),row= 3, col= 1)
        fig.add_trace(go.Scatter(line={'dash': 'dash', 'color': "#A75314"}, name="3m prior", legendgroup = "LT Nominal Rates"),row= 3, col= 1)
        fig.add_trace(go.Scatter(line={'dash': 'dot', 'color': '#1C0C91'}, name="6m prior", legendgroup="Inflation Exepctations"),row= 1, col= 1)
        fig.add_trace(go.Scatter(line={'dash': 'dot', 'color': "#945F8F"}, name="6m prior", legendgroup="Real Rates"),row= 2, col= 1)
        fig.add_trace(go.Scatter(line={'dash': 'dot', 'color': "#692924"}, name="6m prior", legendgroup = "ST Nominal Rates"),row= 3, col= 1)
        fig.add_trace(go.Scatter(line={'dash': 'dot', 'color': "#71390F"}, name="6m prior", legendgroup = "LT Nominal Rates"),row= 3, col= 1)
        
    elif i>= 3 and i < 6:
        frames.append(go.Frame(data = [
            go.Scatter(x=inflation_years, y=inflation_curves.loc[ix, :].values, name="Current Inflation Expectations"),
            go.Scatter(x=real_rate_x, y=real_rates.loc[ix, :].values, name="Real Rates Curve", xaxis='x2'),
            go.Scatter(x=treasury_bonds_x[:-1], y=implied_real_rates.loc[ix, :].values, name="Implied Real Rates Curve", xaxis='x2'),
            go.Scatter(x=treasury_bills_x, y=treasury_bills.loc[ix, :].values, name="Treasury Bill Curve"), 
            go.Scatter(x=treasury_bonds_x, y=treasury_bonds.loc[ix, :].values, name="Treasury Bond Curve"),
            go.Scatter(x=inflation_years, y=inflation_curves.loc[ix-pd.DateOffset(months=3), :].values, name="3m prior"),
            go.Scatter(x=real_rate_x, y=real_rates.loc[ix-pd.DateOffset(months=3), :].values, name="3m prior", xaxis='x2'),
            go.Scatter(x=treasury_bills_x, y=treasury_bills.loc[ix-pd.DateOffset(months=3), :].values, name="3m prior"), 
            go.Scatter(x=treasury_bonds_x, y=treasury_bonds.loc[ix-pd.DateOffset(months=3), :].values, name="3m prior"),
        ]))
        # frames.append(go.Frame(data = go.Scatter(x=inflation_years, y=inflation_curves.loc[ix-pd.DateOffset(months=3), :].values, name="3 months prior"), name=ix.strftime('%d-%b-%Y')))
        # frames.append(go.Scatter(x=inflation_years, y=inflation_curves.loc[ix, :].values, name="3 Months Lag"))
    elif i >= 6:
        frames.append(go.Frame(data = [
            go.Scatter(x=inflation_years, y=inflation_curves.loc[ix, :].values, name="Current Inflation Expectations"), 
            go.Scatter(x=real_rate_x, y=real_rates.loc[ix, :].values, name="Real Rates Curve", xaxis='x2'),
            go.Scatter(x=treasury_bonds_x[:-1], y=implied_real_rates.loc[ix, :].values, name="Implied Real Rates Curve", xaxis='x2'),
            go.Scatter(x=treasury_bills_x, y=treasury_bills.loc[ix, :].values, name="Treasury Bill Curve"), 
            go.Scatter(x=treasury_bonds_x, y=treasury_bonds.loc[ix, :].values, name="Treasury Bond Curve"),
            go.Scatter(x=inflation_years, y=inflation_curves.loc[ix-pd.DateOffset(months=3), :].values, name="3m prior"),
            go.Scatter(x=real_rate_x, y=real_rates.loc[ix-pd.DateOffset(months=3), :].values, name="3m prior", xaxis='x2'),
            go.Scatter(x=treasury_bills_x, y=treasury_bills.loc[ix-pd.DateOffset(months=3), :].values, name="3m prior"), 
            go.Scatter(x=treasury_bonds_x, y=treasury_bonds.loc[ix-pd.DateOffset(months=3), :].values, name="3m prior"),
            go.Scatter(x=inflation_years, y=inflation_curves.loc[ix-pd.DateOffset(months=6), :].values, name="6 months prior"),
            go.Scatter(x=real_rate_x, y=real_rates.loc[ix-pd.DateOffset(months=6), :].values, name="6 months prior", xaxis='x2'),
            go.Scatter(x=treasury_bills_x, y=treasury_bills.loc[ix-pd.DateOffset(months=6), :].values, name="6m prior"), 
            go.Scatter(x=treasury_bonds_x, y=treasury_bonds.loc[ix-pd.DateOffset(months=6), :].values, name="6m prior"),
            ],
            layout=newLayout, name=ix.strftime('%d-%b-%Y')))
    else:
        frames.append(go.Frame(data = [
            go.Scatter(x=inflation_years, y=inflation_curves.loc[ix, :].values, name="Current Inflation Expectations"), 
            go.Scatter(x=real_rate_x, y=real_rates.loc[ix, :].values, name="Real Rates Curve", xaxis='x2'),
            go.Scatter(x=treasury_bonds_x[:-1], y=implied_real_rates.loc[ix, :].values, name="Implied Real Rates Curve", xaxis='x2'),
            go.Scatter(x=treasury_bills_x, y=treasury_bills.loc[ix, :].values, name="Treasury Bill Curve"), 
            go.Scatter(x=treasury_bonds_x, y=treasury_bonds.loc[ix, :].values, name="Treasury Bond Curve"),
            ],
            layout=newLayout, name=ix.strftime('%d-%b-%Y')))
        # frames.append(go.Frame(data = go.Bar(x=spot_subset.index, y=spot_subset["CORESTICKM158SFRBATL"].values, name="Core Median inflation", xaxis='x2'), name=ix.strftime('%d-%b-%Y')))
    slider_step = {"args": [
        [ix.strftime('%d-%b-%Y')],
        {"frame": {"duration": 100, "redraw": True},
         "mode": "immediate",
         "transition": {"duration": 100}}
    ],
        "label": ix.strftime('%d-%b-%Y'),
        "method": "animate"}
    sliders_dict["steps"].append(slider_step)
# fig.add_trace(go.Scatter(x=us_policy_rates.index, y=us_policy_rates[ 'DFF'].values, name="DFF", marker_color = '#A43E35'),row= 3, col= 1)
# fig.add_trace(go.Scatter(x=us_policy_rates.index, y=us_policy_rates[ 'EFFR'].values, name="EFFR", marker_color =  "#FF5F50"), row = 3, col=1)
fig.update_layout(sliders = [sliders_dict])

# fig.layout = layout
fig.frames = frames
# fig = go.Figure(data, layout, frames)
fig

<a id="rates_curves"></a>
## Observations of Yield Curve 

# Treasury Spreads and Flys

In [ ]:
# Taking a look at Treasury Yield Curve
yld_curve = pd.DataFrame()
#row 1
yld_curve['1s2s'] = treasury_bonds['DGS2'] - treasury_bonds['DGS1']
yld_curve['1s5s'] = treasury_bonds['DGS5'] - treasury_bonds['DGS1']
yld_curve['1s10s'] = treasury_bonds['DGS10'] - treasury_bonds['DGS1']
yld_curve['1s20s'] = treasury_bonds['DGS10'] - treasury_bonds['DGS1']
yld_curve['1s30s'] = treasury_bonds['DGS30'] - treasury_bonds['DGS1']

#row 2
yld_curve['2s5s'] = treasury_bonds['DGS5'] - treasury_bonds['DGS2']
yld_curve['2s10s'] = treasury_bonds['DGS10'] - treasury_bonds['DGS2']
yld_curve['2s20s'] = treasury_bonds['DGS20'] - treasury_bonds['DGS2']
yld_curve['2s30s'] = treasury_bonds['DGS30'] - treasury_bonds['DGS2']

#row 3
yld_curve['5s10s'] = treasury_bonds['DGS10'] - treasury_bonds['DGS5']
yld_curve['5s20s'] = treasury_bonds['DGS20'] - treasury_bonds['DGS5']
yld_curve['5s30s'] = treasury_bonds['DGS30'] - treasury_bonds['DGS5']

#row 4
yld_curve['10s20s'] = treasury_bonds['DGS20'] - treasury_bonds['DGS10']
yld_curve['10s30s'] = treasury_bonds['DGS30'] - treasury_bonds['DGS10']
yld_curve['3m10s'] = treasury_bonds['DGS10'] - treasury_bills['DTB3']

#row 5
yld_curve['20s30s'] = treasury_bonds['DGS30'] - treasury_bonds['DGS20']

yld_curve['1s2s5s'] = yld_curve['1s2s'] - yld_curve['2s5s']
yld_curve['1s2s10s'] = yld_curve['1s2s'] - yld_curve['2s10s']
yld_curve['1s2s20s'] = yld_curve['1s2s'] - yld_curve['2s20s']
yld_curve['1s2s30s'] = yld_curve['1s2s'] - yld_curve['2s30s']

yld_curve['1s5s10s'] = yld_curve['1s5s'] - yld_curve['5s10s']
yld_curve['2s5s10s'] = yld_curve['2s5s'] - yld_curve['5s10s']

yld_curve['1s5s20s'] = yld_curve['1s5s'] - yld_curve['5s20s']
yld_curve['2s5s20s'] = yld_curve['2s5s'] - yld_curve['5s20s']

yld_curve['1s5s30s'] = yld_curve['1s5s'] - yld_curve['5s30s']
yld_curve['2s5s30s'] = yld_curve['2s5s'] - yld_curve['5s30s']

yld_curve['1s10s20s'] = yld_curve['1s10s'] - yld_curve['10s20s']
yld_curve['2s10s20s'] = yld_curve['2s10s'] - yld_curve['10s20s']

yld_curve['1s10s30s'] = yld_curve['1s10s'] - yld_curve['10s30s']
yld_curve['2s10s30s'] = yld_curve['2s10s'] - yld_curve['10s30s']

yld_curve['1s20s30s'] = yld_curve['1s20s'] - yld_curve['20s30s']
yld_curve['2s20s30s'] = yld_curve['2s20s'] - yld_curve['20s30s']

yld_curve['5s10s30s'] = yld_curve['5s10s'] - yld_curve['10s30s']
yld_curve['5s20s30s'] = yld_curve['5s20s'] - yld_curve['20s30s']

yld_curve['10s20s30s'] = yld_curve['10s20s'] - yld_curve['10s30s']

yld_curve = yld_curve*100

In [ ]:
sub_list = list(zip(["US Bond Treasuries "+ str(x)+"s Spreads" for x in treasury_bonds_x], ["US Bond Treasuries "+ str(x)+"s Flys" for x in treasury_bonds_x]))
subplot_titles = [name for ix in sub_list for name in ix]
fig = make_subplots(cols=2, rows=6, subplot_titles=subplot_titles, shared_xaxes=True, horizontal_spacing=.06, column_widths=[0.5, 0.5])


fig.update_layout({
    **{'height': 900, 'paper_bgcolor':'white', 'plot_bgcolor' : "white",'yaxis': {'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black", 'range': [-500, 500], 'dtick': 250}, 'title': {'text': "US Bond Treasuries<br>Covering All Available Historical Data"}},
    **{f'yaxis{i}': {'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black", 'range': [-500, 500], 'dtick': 250} for i in range(2, 13)}
    })

color_tracker = {'1': (120, 0, 0), "2": (60, 0, 0), "3m10": (180, 60, 0), "5": (0, 120, 0), "10": (0, 60, 0), "20": (0, 0, 120), "30": (0, 0, 60)}

cols_1 = [col for col in yld_curve.columns if "1s" in col]
cols_2 = [col for col in yld_curve.columns if "2s" in col]
cols_5 = [col for col in yld_curve.columns if "5s" in col]
cols_10 = [col for col in yld_curve.columns if "10s" in col]
cols_20 = [col for col in yld_curve.columns if "20s" in col]
cols_30 = [col for col in yld_curve.columns if "30s" in col]

shown_cols = []

for col in cols_1:
    terms = col.split('s')[:-1]
    color = [90, 90, 90]
    for term in terms:
        for i, x in enumerate(color_tracker[term]):
            color[i] += x
    scatter = go.Scatter(x=yld_curve.index, y=yld_curve[col].values, name=col, legendgroup = col, line={'color': rgb_to_hex(tuple(color))})
    if col in shown_cols:
        scatter.showlegend = False
    else:
        shown_cols.append(col)
    if len(terms) == 2:
        fig.add_trace(scatter, row=1, col=1)
    else:
        fig.add_trace(scatter, row=1, col=2)
    color = [90, 90, 90]
    
for col in cols_2:
    terms = col.split('s')[:-1]
    color = [90, 90, 90]
    for term in terms:
        for i, x in enumerate(color_tracker[term]):
            color[i] += x
    scatter = go.Scatter(x=yld_curve.index, y=yld_curve[col].values, name=col, legendgroup = col, line={'color': rgb_to_hex(tuple(color))})
    if col in shown_cols:
        scatter.showlegend = False
    else:
        shown_cols.append(col)
    if len(terms) == 2:
        fig.add_trace(scatter, row=2, col=1)
    else:
        fig.add_trace(scatter, row=2, col=2)
    color = [90, 90, 90]

for col in cols_5:
    terms = col.split('s')[:-1]
    color = [90, 90, 90]
    for term in terms:
        for i, x in enumerate(color_tracker[term]):
            color[i] += x
    scatter = go.Scatter(x=yld_curve.index, y=yld_curve[col].values, name=col, legendgroup = col, line={'color': rgb_to_hex(tuple(color))})
    if col in shown_cols:
        scatter.showlegend = False
    else:
        shown_cols.append(col)
    if len(terms) == 2:
        fig.add_trace(scatter, row=3, col=1)
    else:
        fig.add_trace(scatter, row=3, col=2)
    color = [90, 90, 90]
    
for col in cols_10:
    terms = col.split('s')[:-1]
    color = [90, 90, 90]
    for term in terms:
        for i, x in enumerate(color_tracker[term]):
            color[i] += x
    scatter = go.Scatter(x=yld_curve.index, y=yld_curve[col].values, name=col, legendgroup = col, line={'color': rgb_to_hex(tuple(color))})
    if col in shown_cols:
        scatter.showlegend = False
    else:
        shown_cols.append(col)
    if len(terms) == 2:
        fig.add_trace(scatter, row=4, col=1)
    else:
        fig.add_trace(scatter, row=4, col=2)
    color = [90, 90, 90]


for col in cols_20:
    terms = col.split('s')[:-1]
    color = [90, 90, 90]
    for term in terms:
        for i, x in enumerate(color_tracker[term]):
            color[i] += x
    scatter = go.Scatter(x=yld_curve.index, y=yld_curve[col].values, name=col, legendgroup = col, line={'color': rgb_to_hex(tuple(color))})
    if col in shown_cols:
        scatter.showlegend = False
    else:
        shown_cols.append(col)
    if len(terms) == 2:
        fig.add_trace(scatter, row=5, col=1)
    else:
        fig.add_trace(scatter, row=5, col=2)
    color = [90, 90, 90]
    
for col in cols_30:
    terms = col.split('s')[:-1]
    color = [90, 90, 90]
    for term in terms:
        for i, x in enumerate(color_tracker[term]):
            color[i] += x
    scatter = go.Scatter(x=yld_curve.index, y=yld_curve[col].values, name=col, legendgroup = col, line={'color': rgb_to_hex(tuple(color))})
    if col in shown_cols:
        scatter.showlegend = False
    else:
        shown_cols.append(col)
    if len(terms) == 2:
        fig.add_trace(scatter, row=6, col=1)
    else:
        fig.add_trace(scatter, row=6, col=2)
    color = [90, 90, 90]

full_axes = {}
black_mon_axes = {}
dotcom_axes = {}
lehman_axes = {}    
trump_axes = {}
present_axes = {}



for i in range(2, 13):
    seed = {}
    if i % 2 != 0:
        seed['domain'] = [0, 0.45]
        seed['matches'] = 'x11'
        
    else:
        seed['domain'] = [.55, 1.0]
        seed['matches'] = 'x12'
    if i >10:
        seed['showticklabels'] = True
        del seed['matches']
    else:
        seed['showticklabels'] = False
        
    full_axes[f'xaxis{i}'] = {**{'range': [yld_curve.index.min(), yld_curve.index.max()], "anchor": f"y{i}"}, **seed}
    black_mon_axes[f'xaxis{i}'] = {**{'range': [pd.Timestamp('1985-01-01'), pd.Timestamp('1990-01-01')], "anchor": f"y{i}"}, **seed}
    dotcom_axes[f'xaxis{i}'] = {**{'range': [pd.Timestamp('1995-04-01'), pd.Timestamp('2000-04-01')], "anchor": f"y{i}"}, **seed}
    lehman_axes[f'xaxis{i}'] = {**{'range': [pd.Timestamp('2000-04-01'), pd.Timestamp('2008-10-01')], "anchor": f"y{i}"},**seed}
    trump_axes[f'xaxis{i}'] = {**{'range': [pd.Timestamp('2008-10-01'), pd.Timestamp('2016-10-01')], "anchor": f"y{i}"}, **seed}
    present_axes[f'xaxis{i}'] = {**{'range': [pd.Timestamp('2016-10-01'), pd.Timestamp('2022-12-31')], "anchor": f"y{i}"}, **seed}

    full_axes[f'yaxis{i}'] = {'range': [-500, 500], 'dtick': 250, 'anchor': fig.layout[f'yaxis{i}']['anchor'], 'domain': fig.layout[f'yaxis{i}']['domain'],'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"}
    black_mon_axes[f'yaxis{i}'] = {'range': [-500, 500], 'dtick': 250, 'anchor': fig.layout[f'yaxis{i}']['anchor'], 'domain': fig.layout[f'yaxis{i}']['domain'], 'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"}
    dotcom_axes[f'yaxis{i}'] = {'range': [-500, 500], 'dtick': 250, 'anchor': fig.layout[f'yaxis{i}']['anchor'], 'domain': fig.layout[f'yaxis{i}']['domain'], 'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"}
    lehman_axes[f'yaxis{i}'] = {'range': [-500, 500], 'dtick': 250, 'anchor': fig.layout[f'yaxis{i}']['anchor'], 'domain': fig.layout[f'yaxis{i}']['domain'], 'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"}
    trump_axes[f'yaxis{i}'] = {'range': [-500, 500], 'dtick': 250, 'anchor': fig.layout[f'yaxis{i}']['anchor'], 'domain': fig.layout[f'yaxis{i}']['domain'], 'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"}
    present_axes[f'yaxis{i}'] = {'range': [-500, 500], 'dtick': 250, 'anchor': fig.layout[f'yaxis{i}']['anchor'], 'domain': fig.layout[f'yaxis{i}']['domain'], 'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"}


fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            active=0,
            x=0.57,
            y=1.2,
            buttons=list([
                dict(label="Full",
                     method="update",
                     args=[{},{**{"xaxis": 
                                  {'range': [yld_curve.index.min(), yld_curve.index.max()], 
                                   'anchor': 'y', 'domain': [0, .45], 'showticklabels': False, 'matches': 'x11'}, 
                            "title": "Covering All Available Historical Data",
                            "yaxis": {'range': [-500, 500], 'dtick': 250, 'anchor': fig.layout['yaxis']['anchor'], 'domain': fig.layout['yaxis']['domain'], 'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"}
                            }, **full_axes}, 
                           [i for i in range(0, 12)]]),
                dict(label="80s",
                     method="update",
                     args=[{},{**{"xaxis": 
                                  {'range': [pd.Timestamp('1985-01-01'), pd.Timestamp('1990-01-01')], 
                                   'anchor': 'y', 'domain': [0, .45], 'showticklabels': False, 'matches': 'x11'}, 
                            "title": "Black Monday Crash to Black Friday Crash",
                            "yaxis": {'range': [-500, 500], 'dtick': 250, 'anchor': fig.layout['yaxis']['anchor'], 'domain': fig.layout['yaxis']['domain'],'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"}
                            }, **black_mon_axes}, 
                           [i for i in range(0, 12)]]),
                dict(label="Dotcom",
                     method="update",
                     args=[{},{**{"xaxis": 
                                  {'range': [pd.Timestamp('1995-01-01'), pd.Timestamp('2000-04-01')], 
                                   'anchor': 'y', 'domain': [0, .45], 'showticklabels': False, 'matches': 'x11'}, 
                            "title": "DotCom Crash Period",
                            "yaxis": {'range': [-500, 500], 'dtick': 250, 'anchor': fig.layout['yaxis']['anchor'], 'domain': fig.layout['yaxis']['domain'],'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"}
                            }, **dotcom_axes}, 
                           [i for i in range(0, 12)]]),
                dict(label="Pre Housing",
                     method="update",
                     args=[{},{**{"xaxis": 
                                  {'range': [pd.Timestamp('2000-04-01'), pd.Timestamp('2008-10-01')], 
                                   'anchor': 'y', 'domain': [0, .45], 'showticklabels': False, 'matches': 'x11'}, 
                            "title": "Post DotCom Crash to Lehman Default",
                            "yaxis": {'range': [-500, 500], 'dtick': 250, 'anchor': fig.layout['yaxis']['anchor'], 'domain': fig.layout['yaxis']['domain'], 'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"}
                            }, **lehman_axes}, 
                            
                           [i for i in range(0, 12)]]),
                dict(label="Pre 2016",
                     method="update",
                     args=[{},{**{"xaxis": {'range': [pd.Timestamp('2008-10-01'), pd.Timestamp('2016-10-01')],
                                            'anchor': 'y', 'domain': [0, .45], 'showticklabels': False, 'matches': 'x11'},
                            "title": "Post Lehman Default to Brexit & Trump Election Beginning",
                            "yaxis": {'range': [-500, 500], 'dtick': 250, 'anchor': fig.layout['yaxis']['anchor'], 'domain': fig.layout['yaxis']['domain'], 'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"}
                            }, **trump_axes}, 
                           [i for i in range(0, 12)]]),
                dict(label="Present",
                     method="update",
                     args=[{},{**{"xaxis": {'range': [pd.Timestamp('2016-10-01'), pd.Timestamp('2022-12-31')],
                                            'anchor': 'y', 'domain': [0, .45], 'showticklabels': False, 'matches': 'x11'},
                            "title": "Post Brexit & Trump Election to Present", 
                            "yaxis": {'range': [-500, 500], 'dtick': 250, 'anchor': fig.layout['yaxis']['anchor'], 'domain': fig.layout['yaxis']['domain'],'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"} 
                            }, **present_axes},
                            
                           [i for i in range(0, 12)]]),

            ]),
        )
    ])
fig

In [ ]:
subplot_titles[:2]

In [ ]:
fig.data[0].name

In [ ]:
sub_list = list(zip(["US Bond Treasuries "+ str(x)+"s Spreads" for x in treasury_bonds_x], ["US Bond Treasuries "+ str(x)+"s Flys" for x in treasury_bonds_x]))
subplot_titles = [name for ix in sub_list for name in ix]
fig = make_subplots(cols=2, rows=1, subplot_titles=subplot_titles[:2], shared_xaxes=True, horizontal_spacing=.06, column_widths=[0.5, 0.5])


fig.update_layout({
    **{'height': 900, 'paper_bgcolor':'white', 'plot_bgcolor' : "white",'yaxis': {'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black", 'range': [-500, 500], 'dtick': 250}, 'title': {'text': "US Bond Treasuries<br>Covering All Available Historical Data"}},
    **{f'yaxis{i}': {'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black", 'range': [-500, 500], 'dtick': 250} for i in range(2, 13)}
    })

color_tracker = {'1': (120, 0, 0), "2": (60, 0, 0), "3m10": (180, 60, 0), "5": (0, 120, 0), "10": (0, 60, 0), "20": (0, 0, 120), "30": (0, 0, 60)}

cols_1 = [col for col in yld_curve.columns if "1s" in col]
cols_2 = [col for col in yld_curve.columns if "2s" in col]
cols_5 = [col for col in yld_curve.columns if "5s" in col]
cols_10 = [col for col in yld_curve.columns if "10s" in col]
cols_20 = [col for col in yld_curve.columns if "20s" in col]
cols_30 = [col for col in yld_curve.columns if "30s" in col]

shown_cols = []


for col in cols_1:
    terms = col.split('s')[:-1]
    color = [90, 90, 90]
    for term in terms:
        for i, x in enumerate(color_tracker[term]):
            color[i] += x
    scatter = go.Scatter(x=yld_curve.index, y=yld_curve[col].values, name=col, legendgroup = col, line={'color': rgb_to_hex(tuple(color))})
    if col in shown_cols:
        scatter.showlegend = False
    else:
        shown_cols.append(col)
    if len(terms) == 2:
        fig.add_trace(scatter, row=1, col=1)
    else:
        fig.add_trace(scatter, row=1, col=2)
    color = [90, 90, 90]


for col in cols_2:
    terms = col.split('s')[:-1]
    color = [90, 90, 90]
    for term in terms:
        for i, x in enumerate(color_tracker[term]):
            color[i] += x
    scatter = go.Scatter(x=yld_curve.index, y=yld_curve[col].values, name=col, legendgroup = col, visible = False, line={'color': rgb_to_hex(tuple(color))})
    if col in shown_cols:
        scatter.showlegend = False
    else:
        shown_cols.append(col)
    if len(terms) == 2:
        fig.add_trace(scatter, row=1, col=1)
    else:
        fig.add_trace(scatter, row=1, col=2)
    color = [90, 90, 90]

for col in cols_5:
    terms = col.split('s')[:-1]
    color = [90, 90, 90]
    for term in terms:
        for i, x in enumerate(color_tracker[term]):
            color[i] += x
    scatter = go.Scatter(x=yld_curve.index, y=yld_curve[col].values, name=col, legendgroup = col, line={'color': rgb_to_hex(tuple(color))})
    if col in shown_cols:
        scatter.showlegend = False
    else:
        shown_cols.append(col)
    if len(terms) == 2:
        fig.add_trace(scatter, row=1, col=1)
    else:
        fig.add_trace(scatter, row=1, col=2)
    color = [90, 90, 90]

    
for col in cols_10:
    terms = col.split('s')[:-1]
    color = [90, 90, 90]
    for term in terms:
        for i, x in enumerate(color_tracker[term]):
            color[i] += x
    scatter = go.Scatter(x=yld_curve.index, y=yld_curve[col].values, name=col, legendgroup = col, visible=False, line={'color': rgb_to_hex(tuple(color))})
    if col in shown_cols:
        scatter.showlegend = False
    else:
        shown_cols.append(col)
    if len(terms) == 2:
        fig.add_trace(scatter, row=1, col=1)
    else:
        fig.add_trace(scatter, row=1, col=2)
    color = [90, 90, 90]



for col in cols_20:
    terms = col.split('s')[:-1]
    color = [90, 90, 90]
    for term in terms:
        for i, x in enumerate(color_tracker[term]):
            color[i] += x
    scatter = go.Scatter(x=yld_curve.index, y=yld_curve[col].values, name=col, legendgroup = col, line={'color': rgb_to_hex(tuple(color))})
    if col in shown_cols:
        scatter.showlegend = False
    else:
        shown_cols.append(col)
    if len(terms) == 2:
        fig.add_trace(scatter, row=1, col=1)
    else:
        fig.add_trace(scatter, row=1, col=2)
    color = [90, 90, 90]

    
for col in cols_30:
    terms = col.split('s')[:-1]
    color = [90, 90, 90]
    for term in terms:
        for i, x in enumerate(color_tracker[term]):
            color[i] += x
    scatter = go.Scatter(x=yld_curve.index, y=yld_curve[col].values, name=col, legendgroup = col, line={'color': rgb_to_hex(tuple(color))})
    if col in shown_cols:
        scatter.showlegend = False
    else:
        shown_cols.append(col)
    if len(terms) == 2:
        fig.add_trace(scatter, row=1, col=1)
    else:
        fig.add_trace(scatter, row=1, col=2)
    color = [90, 90, 90]


full_axes = {}
black_mon_axes = {}
dotcom_axes = {}
lehman_axes = {}    
trump_axes = {}
present_axes = {}



for i in range(2, 3):
    seed = {}
    if i % 2 != 0:
        seed['domain'] = [0, 0.45]
        seed['matches'] = 'x11'
        
    else:
        seed['domain'] = [.55, 1.0]
        seed['matches'] = 'x12'
    if i >10:
        seed['showticklabels'] = True
        del seed['matches']
    else:
        seed['showticklabels'] = False
        
    full_axes[f'xaxis{i}'] = {**{'range': [yld_curve.index.min(), yld_curve.index.max()], "anchor": f"y{i}"}, **seed}
    black_mon_axes[f'xaxis{i}'] = {**{'range': [pd.Timestamp('1985-01-01'), pd.Timestamp('1990-01-01')], "anchor": f"y{i}"}, **seed}
    dotcom_axes[f'xaxis{i}'] = {**{'range': [pd.Timestamp('1995-04-01'), pd.Timestamp('2000-04-01')], "anchor": f"y{i}"}, **seed}
    lehman_axes[f'xaxis{i}'] = {**{'range': [pd.Timestamp('2000-04-01'), pd.Timestamp('2008-10-01')], "anchor": f"y{i}"},**seed}
    trump_axes[f'xaxis{i}'] = {**{'range': [pd.Timestamp('2008-10-01'), pd.Timestamp('2016-10-01')], "anchor": f"y{i}"}, **seed}
    present_axes[f'xaxis{i}'] = {**{'range': [pd.Timestamp('2016-10-01'), pd.Timestamp('2022-12-31')], "anchor": f"y{i}"}, **seed}

    full_axes[f'yaxis{i}'] = {'range': [-500, 500], 'dtick': 250, 'anchor': fig.layout[f'yaxis{i}']['anchor'], 'domain': fig.layout[f'yaxis{i}']['domain'],'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"}
    black_mon_axes[f'yaxis{i}'] = {'range': [-500, 500], 'dtick': 250, 'anchor': fig.layout[f'yaxis{i}']['anchor'], 'domain': fig.layout[f'yaxis{i}']['domain'], 'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"}
    dotcom_axes[f'yaxis{i}'] = {'range': [-500, 500], 'dtick': 250, 'anchor': fig.layout[f'yaxis{i}']['anchor'], 'domain': fig.layout[f'yaxis{i}']['domain'], 'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"}
    lehman_axes[f'yaxis{i}'] = {'range': [-500, 500], 'dtick': 250, 'anchor': fig.layout[f'yaxis{i}']['anchor'], 'domain': fig.layout[f'yaxis{i}']['domain'], 'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"}
    trump_axes[f'yaxis{i}'] = {'range': [-500, 500], 'dtick': 250, 'anchor': fig.layout[f'yaxis{i}']['anchor'], 'domain': fig.layout[f'yaxis{i}']['domain'], 'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"}
    present_axes[f'yaxis{i}'] = {'range': [-500, 500], 'dtick': 250, 'anchor': fig.layout[f'yaxis{i}']['anchor'], 'domain': fig.layout[f'yaxis{i}']['domain'], 'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"}


fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            active=0,
            x=0.57,
            y=1.2,
            buttons=list([
                dict(label="Full",
                     method="update",
                     args=[{},{**{"xaxis": 
                                  {'range': [yld_curve.index.min(), yld_curve.index.max()], 
                                   'anchor': 'y', 'domain': [0, .45], 'showticklabels': False, 'matches': 'x11'}, 
                            "title": "Covering All Available Historical Data",
                            "yaxis": {'range': [-500, 500], 'dtick': 250, 'anchor': fig.layout['yaxis']['anchor'], 'domain': fig.layout['yaxis']['domain'], 'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"}
                            }, **full_axes}, 
                           [i for i in range(0, 2)]]),
                dict(label="80s",
                     method="update",
                     args=[{},{**{"xaxis": 
                                  {'range': [pd.Timestamp('1985-01-01'), pd.Timestamp('1990-01-01')], 
                                   'anchor': 'y', 'domain': [0, .45], 'showticklabels': False, 'matches': 'x11'}, 
                            "title": "Black Monday Crash to Black Friday Crash",
                            "yaxis": {'range': [-500, 500], 'dtick': 250, 'anchor': fig.layout['yaxis']['anchor'], 'domain': fig.layout['yaxis']['domain'],'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"}
                            }, **black_mon_axes}, 
                           [i for i in range(0, 2)]]),
                dict(label="Dotcom",
                     method="update",
                     args=[{},{**{"xaxis": 
                                  {'range': [pd.Timestamp('1995-01-01'), pd.Timestamp('2000-04-01')], 
                                   'anchor': 'y', 'domain': [0, .45], 'showticklabels': False, 'matches': 'x11'}, 
                            "title": "DotCom Crash Period",
                            "yaxis": {'range': [-500, 500], 'dtick': 250, 'anchor': fig.layout['yaxis']['anchor'], 'domain': fig.layout['yaxis']['domain'],'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"}
                            }, **dotcom_axes}, 
                           [i for i in range(0, 2)]]),
                dict(label="Pre Housing",
                     method="update",
                     args=[{},{**{"xaxis": 
                                  {'range': [pd.Timestamp('2000-04-01'), pd.Timestamp('2008-10-01')], 
                                   'anchor': 'y', 'domain': [0, .45], 'showticklabels': False, 'matches': 'x11'}, 
                            "title": "Post DotCom Crash to Lehman Default",
                            "yaxis": {'range': [-500, 500], 'dtick': 250, 'anchor': fig.layout['yaxis']['anchor'], 'domain': fig.layout['yaxis']['domain'], 'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"}
                            }, **lehman_axes}, 
                            
                           [i for i in range(0, 2)]]),
                dict(label="Pre 2016",
                     method="update",
                     args=[{},{**{"xaxis": {'range': [pd.Timestamp('2008-10-01'), pd.Timestamp('2016-10-01')],
                                            'anchor': 'y', 'domain': [0, .45], 'showticklabels': False, 'matches': 'x11'},
                            "title": "Post Lehman Default to Brexit & Trump Election Beginning",
                            "yaxis": {'range': [-500, 500], 'dtick': 250, 'anchor': fig.layout['yaxis']['anchor'], 'domain': fig.layout['yaxis']['domain'], 'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"}
                            }, **trump_axes}, 
                           [i for i in range(0, 2)]]),
                dict(label="Present",
                     method="update",
                     args=[{},{**{"xaxis": {'range': [pd.Timestamp('2016-10-01'), pd.Timestamp('2022-12-31')],
                                            'anchor': 'y', 'domain': [0, .45], 'showticklabels': False, 'matches': 'x11'},
                            "title": "Post Brexit & Trump Election to Present", 
                            "yaxis": {'range': [-500, 500], 'dtick': 250, 'anchor': fig.layout['yaxis']['anchor'], 'domain': fig.layout['yaxis']['domain'],'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"} 
                            }, **present_axes},
                            
                           [i for i in range(0, 2)]]),

            ]),
        ),
        dict(
            type="buttons",
            buttons=[
                dict(label="1s",
                     method="update",
                     args=[{
                         'visible': vis_1s
                     }, {}]),
                dict(label="2s",
                     method="update",
                     args=[{
                         'visible': vis_2s
                         
                     },{}]),
                dict(label="5s",
                     method="update",
                     args=[{
                         'visible': vis_5s
                         
                     },{}]),
                dict(label="10s",
                     method="update",
                     args=[{
                         'visible': vis_10s
                         
                     },{}]),
                dict(label="20s",
                     method="update",
                     args=[{
                         'visible': vis_20s
                         
                     },{}]),
                
            ],
        )
    ]
)
fig

## Hint

Better to focus on a few series by first double clicking on 1 and then adding a few comparables by single clicking on others.


In [ ]:
px.line(us_policy_rates.ffill(), title="Fed Policy Rates Historical")


# Hint

Left Click and Drag to draw a box around the time period of interest to get a closer look

# Observations

The charts measuring the flys suggest we are at a period of overall flattness in the US Treasury Bond curve and we have not reached the level of inversion seen since the 80s (sparked after period caused by the crash in [Black Monday](https://en.wikipedia.org/wiki/Black_Monday_(1987))) suggesting that we are approaching a key pivot point. Black Monday is important (please click the 80s tab) because you will observe that the 10s20s vs 20s30s treasury relationship exhibited an important trading relationship commonly seen during distressed markets - when markets the distressed, people sell what is most liquid - not necessarily what is percieved to be most expensive on the curve. 20y has historically had less liquidity than the 10y (the flight to safety asset) and 30y (the mortgage hedge) so in a period of deep uncertainty when rates moved very quickly and there was a period of high systemic risk, it's important to note that 10s30s hardly moved as bonds were moving in a 300bp range.

#### Assessment

The curve is currently slightly inverted and can worsen in my opinion if the situation in Russia and Ukraine worsens, a new risk emerges with China and Taiwan tensions flaring or some natural disaster / geopolitical unforseen risk / biological risk somehow greatly upsets global trade barely forming repaired routes then curve can invert signifcantly more if this percieved as a near term risk. However, in event of a positive growth/inflation outcome (ie we see more reason for Fed thinking gdp hasn't been too affected by Rate Hikes in Q3 - they started in May so it would affect this quarter) I believe there can still be large hikes of 75bps in the horizon for Nov (otherwise I'd imagine they start easing the gas to 50bps).

##### Key Risks
* If there is long term growth risk?
* severe unemployment, humanitarian concerns?

[Return Home](#tbc)


In [ ]:
frames = []

sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Year:",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}


fig = make_subplots(rows=3,cols= 1, subplot_titles = ("Inflation Expecations", "Real Rates", "Fed Policy Rate"))
fig.update_layout(height = 700, paper_bgcolor='white', plot_bgcolor = "white", updatemenus=[dict(type="buttons", buttons=[   {
                "args": [None, {"frame": {"duration": 500, "redraw": False},
                                "fromcurrent": True, "transition": {"duration": 300,
                                                                    "easing": "quadratic-in-out"}}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        ])], yaxis={'range':[-1, 8], 'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"}, yaxis2={'range':[-10, 12], 'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': 'black'}, 
        yaxis3={'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"}, title={'text': "Future Inflation Expectations vs Real Rates vs Fed Policy Rates"},
        barmode='group', bargap=.1)

for i, ix in enumerate(real_rates.index):
    newLayout = fig.layout
    fig.update_layout(xaxis3={'range': [ix - pd.DateOffset(months=3),ix]})
    # newLayout.yaxis['range'] = [inflation_curves.loc[ix, :].min()-1.5, inflation_curves.loc[ix, :].max() + 1.5]
    if i == 0:
        fig.add_trace(go.Scatter(x=inflation_years, y=inflation_curves.loc[ix, :].values, name="Current Inflation Expectations", line={'color': '#9CC6DE'}),row= 1, col= 1)
        fig.add_trace(go.Scatter(x=real_rate_x, y=real_rates.loc[ix, :].values, name="Real Rates Curve", xaxis='x2', line={'color': "#DEB6DA"}), row=2, col=1)
        fig.add_trace(go.Scatter(line={'dash': 'dash', 'color': '#3116F5'}, name="3 months prior"),row= 1, col= 1)
        fig.add_trace(go.Scatter(line={'dash': 'dot', 'color': '#1C0C91'}, name="6 months prior"),row= 1, col= 1)
        fig.add_trace(go.Scatter(line={'dash': 'dash', 'color': "#DE8ED6"}, name="3 months prior"),row= 2, col= 1)
        fig.add_trace(go.Scatter(line={'dash': 'dot', 'color': "#945F8F"}, name="6 months prior"),row= 2, col= 1)
        
    elif i>= 3 and i < 6:
        frames.append(go.Frame(data = [
            go.Scatter(x=inflation_years, y=inflation_curves.loc[ix, :].values, name="Current Inflation Expectations"),
            go.Scatter(x=real_rate_x, y=real_rates.loc[ix, :].values, name="Real Rates Curve", xaxis='x2'), 
            go.Scatter(x=inflation_years, y=inflation_curves.loc[ix-pd.DateOffset(months=3), :].values, name="3 months prior"),
            go.Scatter(),
            go.Scatter(x=real_rate_x, y=real_rates.loc[ix-pd.DateOffset(months=3), :].values, name="3 months prior", xaxis='x2')]))
        # frames.append(go.Frame(data = go.Scatter(x=inflation_years, y=inflation_curves.loc[ix-pd.DateOffset(months=3), :].values, name="3 months prior"), name=ix.strftime('%d-%b-%Y')))
        # frames.append(go.Scatter(x=inflation_years, y=inflation_curves.loc[ix, :].values, name="3 Months Lag"))
    elif i >= 6:
        frames.append(go.Frame(data = [
            go.Scatter(x=inflation_years, y=inflation_curves.loc[ix, :].values, name="Current Inflation Expectations"), 
            go.Scatter(x=real_rate_x, y=real_rates.loc[ix, :].values, name="Real Rates Curve", xaxis='x2'),  
            go.Scatter(x=inflation_years, y=inflation_curves.loc[ix-pd.DateOffset(months=3), :].values, name="3 months prior"),
            go.Scatter(x=inflation_years, y=inflation_curves.loc[ix-pd.DateOffset(months=6), :].values, name="6 months prior"),
            go.Scatter(x=real_rate_x, y=real_rates.loc[ix-pd.DateOffset(months=3), :].values, name="3 months prior", xaxis='x2'),
            go.Scatter(x=real_rate_x, y=real_rates.loc[ix-pd.DateOffset(months=6), :].values, name="6 months prior", xaxis='x2'),
            ],
            layout=newLayout, name=ix.strftime('%d-%b-%Y')))
    else:
        frames.append(go.Frame(data = [
            go.Scatter(x=inflation_years, y=inflation_curves.loc[ix, :].values, name="Current Inflation Expectations"), 
            go.Scatter(x=real_rate_x, y=real_rates.loc[ix, :].values, name="Real Rates Curve", xaxis='x2'),
            ],
            layout=newLayout, name=ix.strftime('%d-%b-%Y')))
        # frames.append(go.Frame(data = go.Bar(x=spot_subset.index, y=spot_subset["CORESTICKM158SFRBATL"].values, name="Core Median inflation", xaxis='x2'), name=ix.strftime('%d-%b-%Y')))
    slider_step = {"args": [
        [ix.strftime('%d-%b-%Y')],
        {"frame": {"duration": 100, "redraw": True},
         "mode": "immediate",
         "transition": {"duration": 100}}
    ],
        "label": ix.strftime('%d-%b-%Y'),
        "method": "animate"}
    sliders_dict["steps"].append(slider_step)
fig.add_trace(go.Scatter(x=us_policy_rates.index, y=us_policy_rates[ 'DFF'].values, name="DFF", marker_color = '#FF5F50'),row= 3, col= 1)
fig.add_trace(go.Scatter(x=us_policy_rates.index, y=us_policy_rates[ 'EFFR'].values, name="EFFR", marker_color =  "#A43E35"), row = 3, col=1)
fig.update_layout(sliders = [sliders_dict])

# fig.layout = layout
fig.frames = frames
# fig = go.Figure(data, layout, frames)
fig

## Observations

Inflation expecations seem to have been anchored low from 2008 onwwards which has allowed policy makers to keep rates at closer to zero and enjoy low inflation expectations despite having negative/zero real rates and previously higher inflation (in 2016-2020 period). 

Story changes in late 2019 when real rates take a significant dive lower and policy rates are forced would normally have been forced higher to offset inflation risks.

[Return Home](#tbc)

<a id=gdp_intro></a>

# Real GDP & GNI visuals


In [ ]:
gdp = web.DataReader(["A191RL1Q225SBEA", "A261RL1Q225SBEA"], 'fred', datetime(1970, 6, 1), datetime.now())
gdp.columns = ['Real_GDP_QoQ', "Real_GNI_QoQ"]
gdp_mapper = {1: "Q1", 4: "Q2", 7: "Q3", 10: "Q4"}
gdp['quarter'] = gdp.index.month.map(gdp_mapper)
gdp['gdp_text'] = gdp['quarter'] + " "+ gdp.index.year.astype('str') + "\n Real GDP: "+ gdp['Real_GDP_QoQ'].astype('str')
gdp['gni_text'] = gdp['quarter'] + " "+ gdp.index.year.astype('str') + "\n Real GNI: "+ gdp['Real_GNI_QoQ'].astype('str')

In [ ]:
fig = make_subplots(1, 1)


fig.add_trace(go.Scatter(x=gdp.index, y=gdp['Real_GDP_QoQ'].values, text=gdp['gdp_text'].values, name =  "Real GDP", marker_color = "magenta", mode= 'lines+markers'))
fig.add_trace(go.Scatter(x=gdp.index, y=gdp['Real_GNI_QoQ'].values, text=gdp['gni_text'].values, name =  "Real GNI", marker_color = "darkviolet", mode= 'lines+markers'))
for quarter in gdp_mapper.values():
    gdp_quarter = gdp.loc[gdp.quarter == quarter, :]
    fig.add_trace(go.Scatter(x=gdp_quarter.index, y=gdp_quarter['Real_GDP_QoQ'].values, text=gdp_quarter['gdp_text'].values, name =  f"Real GDP - {quarter}", legendgroup = "Real GDP", marker_color = "magenta", visible=False, mode= 'lines+markers'))
    fig.add_trace(go.Scatter(x=gdp_quarter.index, y=gdp_quarter['Real_GNI_QoQ'].values, text=gdp_quarter['gni_text'].values, name =  f"Real GNI - {quarter}", legendgroup = "Real GNI", marker_color = "darkviolet", visible=False, mode= 'lines+markers'))
fig.update_layout({
    'height': 900, 'paper_bgcolor':'white', 'plot_bgcolor' : "white",
    'yaxis': {'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black", 
              'title': {'text': "3m/3m Rate "}
             }, 
    'title': {'text': "US Real GDP / GNI<br>Covering All Available Historical Data"}
}
    )
all_vis = []
q1_vis = []
q2_vis = []
q3_vis = []
q4_vis = []
for plt in fig.data:
    splits = plt.name.split(' - ') 
    if len(splits)== 2:
        all_vis.append(False)
        if splits[1] == "Q1":
            q1_vis.append(True)
        else:
            q1_vis.append(False)
            
        if splits[1] == "Q2":
            q2_vis.append(True)
        else:
            q2_vis.append(False)
        
        if splits[1] == "Q3":
            q3_vis.append(True)
        else:
            q3_vis.append(False)
            
        if splits[1] == "Q4":
            q4_vis.append(True)
        else:
            q4_vis.append(False)
    else:
        all_vis.append(True)
        q1_vis.append(False)
        q2_vis.append(False)
        q3_vis.append(False)
        q4_vis.append(False)

fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            active=0,
            x=0.57,
            y=1.2,
            buttons=list([
                dict(label="Full",
                     method="update",
                     args=[{},{"xaxis": 
                                  {'range': [yld_curve.index.min(), yld_curve.index.max()] 
                                   }, 
                            "title": "US Real GDP / GNI<br>Covering All Available Historical Data",
                            "yaxis": {'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"}
                            }, 
                           [0]]),
                dict(label="80s",
                     method="update",
                     args=[{},{"xaxis": 
                                  {'range': [pd.Timestamp('1985-01-01'), pd.Timestamp('1990-01-01')] 
                                   }, 
                            "title": "Black Monday Crash to Black Friday Crash",
                            "yaxis": {'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"}
                            }, 
                           [0]]),
                dict(label="Dotcom",
                     method="update",
                     args=[{},{"xaxis": 
                                  {'range': [pd.Timestamp('1995-01-01'), pd.Timestamp('2000-04-01')], 
                                   }, 
                            "title": "DotCom Crash to Lehman Default",
                            "yaxis": {'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"}
                            }, 
                           [0]]),
                dict(label="Pre Housing",
                     method="update",
                     args=[{},{"xaxis": 
                                  {'range': [pd.Timestamp('2000-04-01'), pd.Timestamp('2008-10-01')], 
                                   }, 
                            "title": "DotCom Crash to Lehman Default",
                            "yaxis": {'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"}
                            }, 
                            
                           [0]]),
                dict(label="Pre 2016",
                     method="update",
                     args=[{}, {"xaxis": {'range': [pd.Timestamp('2008-10-01'), pd.Timestamp('2016-10-01')],
                                        },
                            "title": "Lehman Default to Brexit & Trump Election Beginning",
                            "yaxis": {'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"}
                            }, 
                           [0]]),
                dict(label="Present",
                     method="update",
                     args=[{},{"xaxis": {'range': [pd.Timestamp('2016-10-01'), pd.Timestamp('2022-12-31')],
                                        },
                            "title": "Post Brexit & Trump Election to Presnet", 
                            "yaxis": {'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"} 
                            },
                            
                           [0]]),

            ]),
        ),
        dict(
            type="buttons",
            buttons=[
                dict(label="All",
                     method="update",
                     args=[{
                         'visible': all_vis
                     }, {}]),
                dict(label="Q1 Only",
                     method="update",
                     args=[{
                         'visible': q1_vis
                         
                     },{}]),
                dict(label="Q2 Only",
                     method="update",
                     args=[{
                         'visible': q2_vis
                         
                     },{}]),
                dict(label="Q3 Only",
                     method="update",
                     args=[{
                         'visible': q3_vis
                         
                     },{}]),
                dict(label="Q4 Only",
                     method="update",
                     args=[{
                         'visible': q4_vis
                         
                     },{}]),
                
            ],
        )
    ]
)
        
   

## Observations

Real GDP has been negative so far this year at -1.6% and -0.6% and we have a median forecast of recovery to 2.3%. Will check how the Time weries projections look on this coming soon but I imagine that regressions incorporating the real value of the USD and employment data from job databases will reveal sharp slow downs.

Looking at the visual through the cross cuts of only certain quarters gives me the impression that there is a period of mean reversion across quarters so will be looking at correlation plots across each quarter as well as the whole data set.


[Return Home](#tbc)

<a id="gdp_ts_acf"></a>
# Time Series Models - Partial / Autocorrelation Plots

Following Time Series predictions will later be used as input to a more sophisticated model - for now, will try to fine tune the hypter parameters to the best of my ability on GDP


In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
real_gdp = plot_acf(gdp['Real_GDP_QoQ'].head(50), title="Real GDP Autocorrelation Plot")

In [ ]:
real_gdp_plot = plot_pacf(gdp['Real_GDP_QoQ'].head(50), title="Real GDP Partial Autocorrelation Plot")

## Observations on Autocorrelation Plots for Full GDP Series

There seems to be a point of significance (negative correlation) on the PACF at 12 years (very close at 8 years and 1y) for the AR model though no points of signficance on the ACF plot suggesting not a great candidate for the MA model.

## Autocorrelation plots for each quarter
[Return Home](#tbc)

In [ ]:
q1 = plot_acf(gdp.loc[gdp.quarter == 'Q1',['Real_GDP_QoQ']].head(30))

In [ ]:
q2 = plot_acf(gdp.loc[gdp.quarter == 'Q2',['Real_GDP_QoQ']].head(30))

In [ ]:
q3 = plot_acf(gdp.loc[gdp.quarter == 'Q3',['Real_GDP_QoQ']].head(30))

In [ ]:
q4 = plot_acf(gdp.loc[gdp.quarter == 'Q4',['Real_GDP_QoQ']].head(30))

<a id='realgdp_autocorrelation'></a>
## Observations on Autocorrelation plots
 - Across all quarters, there are periods  of significance for Q2 and close to significance for other quarters around 2-3 years which gives a sliver of hope that MA models might work here.
 
[Return Home](#tbc)

In [ ]:
q1plot = plot_pacf(gdp.loc[gdp.quarter == 'Q1',['Real_GDP_QoQ']].head(50))

In [ ]:
q2plot = plot_pacf(gdp.loc[gdp.quarter == 'Q2',['Real_GDP_QoQ']].head(50))

In [ ]:
q3_plot = plot_pacf(gdp.loc[gdp.quarter == 'Q3',['Real_GDP_QoQ']].head(50))

In [ ]:
q4plot = plot_pacf(gdp.loc[gdp.quarter == 'Q4',['Real_GDP_QoQ']].head(50))

## Observations of PACF in individual quarters

- There seems to be some evidence of stronger Partial autocorrelation in Q1

[Return Home](#tbc)

<a id="gdp_ts_forecast"></a>
# Time Series forecasts of GDP


In [ ]:
predictions = []
predictions2 = []
predictions3 = []
blended_prediction = []
ix_list = []

        
for i in range(14, len(gdp['Real_GDP_QoQ'])+1):
    model = ARIMA(gdp['Real_GDP_QoQ'].head(i), order=(12, 0, 0)).fit()
    model2 = ARIMA(gdp['Real_GDP_QoQ'].head(i), order=(8, 0, 2)).fit()
    model3 = ARIMA(gdp['Real_GDP_QoQ'].head(i), order=(1, 0, 0)).fit()
    predictions.append(model.forecast()[0])
    predictions2.append(model2.forecast()[0])
    predictions3.append(model3.forecast()[0])
    blended_prediction.append(model3.forecast()[0]*.1+model2.forecast()[0]*.2+model.forecast()[0]*.7)
    ix_list.append(model.forecast().index[0])
blended_prediction2 = [e*.7+predictions3[i]*.3 for i,e in enumerate(predictions)]

In [ ]:
orders = {
    "Q1": (13,0,0),
    "Q2": (0,0,3),
    "Q3": (12,0,0),
    "Q4": (11,0,0)}

q_ix = {}
quarter_predictions = {}
for quarter in ['Q1', 'Q2', 'Q3', 'Q4']:
    for i in range(14, len(gdp.loc[gdp.quarter == quarter, 'Real_GDP_QoQ'])+1):
        model = ARIMA(gdp.loc[gdp.quarter == quarter, 'Real_GDP_QoQ'].head(i), order=orders[quarter]).fit()
        try:
            quarter_predictions[quarter].append(model.forecast()[0])
            q_ix[quarter].append(model.forecast().index[0])
        except:
            quarter_predictions[quarter] = [model.forecast()[0]]
            q_ix[quarter] = [model.forecast().index[0]]

In [ ]:
model_arch_1 = arch_model(gdp.loc[gdp.quarter == quarter, 'Real_GDP_QoQ'].head(i), vol="Garch", p=1, o=0, q=1, dist="Normal")

In [ ]:
res = model_arch_1.fit(update_freq = 5)

In [ ]:
forecasts = res.forecast(reindex=False)

In [ ]:
forecasts.mean

In [ ]:
predictions_df = pd.DataFrame({'ix': ix_list, "predict1": predictions, "predict2": predictions2, "predict3": predictions3})
predictions_df.set_index('ix', inplace=True)
gdp_new = pd.concat([gdp, predictions_df], axis=1)

In [ ]:
fig = make_subplots(1, 1)


fig.add_trace(go.Scatter(x=gdp.index, y=gdp_new['Real_GDP_QoQ'].values, text=gdp_new['gdp_text'].values, name =  "Real GDP", marker_color = "magenta", mode= 'lines+markers'))
fig.add_trace(go.Scatter(x=ix_list, y=predictions, name =  "AR(12) Predicted Value", marker_color = "mediumvioletred", mode= 'lines+markers', line={'dash':'longdash'}))
fig.add_trace(go.Scatter(x=ix_list, y=predictions2, name =  "ARMA(8,2) Predicted Value", marker_color = "fuchsia", mode= 'lines+markers', line={'dash':'dashdot'}))
fig.add_trace(go.Scatter(x=ix_list, y=predictions3, name =  "AR(1) Predicted Value", marker_color = "burlywood", mode= 'lines+markers', line={'dash':'dash'}))
fig.add_trace(go.Scatter(x=ix_list, y=blended_prediction, name =  "Blended Prediction", marker_color = "lightpink", mode= 'lines+markers'))


for quarter in gdp_mapper.values():
    gdp_quarter = gdp_new.loc[gdp_new.quarter == quarter, :]
    fig.add_trace(go.Scatter(x=gdp_quarter.index, y=gdp_quarter['Real_GDP_QoQ'].values, text=gdp_quarter['gdp_text'].values, name =  f"Real GDP - {quarter}", legendgroup = "Real GDP", marker_color = "mediumvioletred", visible=False, mode= 'lines+markers'))
    fig.add_trace(go.Scatter(x=gdp_quarter.index, y=gdp_quarter['predict1'].values, name =  f"AR(12) Predicted Value - {quarter}", legendgroup = "Predictions Full", line ={'dash': 'longdash'}, marker_color = "mediumvioletred", visible=False, mode= 'lines+markers'))
    fig.add_trace(go.Scatter(x=gdp_quarter.index, y=gdp_quarter['predict2'].values, name =  f"ARMA(8,2) Predicted Value - {quarter}", legendgroup = "Predictions Full", marker_color = "fuchsia", visible=False, mode= 'lines+markers', line={'dash':'dashdot'}))
    fig.add_trace(go.Scatter(x=gdp_quarter.index, y=gdp_quarter['predict3'].values, name =  f"AR(1) Predicted Value - {quarter}", legendgroup = "Predictions Full", marker_color = "burlywood", visible=False, mode= 'lines+markers', line={'dash':'dash'}))
    fig.add_trace(go.Scatter(x=q_ix[quarter], y=quarter_predictions[quarter], name =  f"ARMA predictions - {quarter}", legendgroup = "Predictions Quarter", marker_color = "blueviolet", visible=False, mode= 'lines+markers', line={'dash':'5px'}))
fig.update_layout({
    'height': 900, 'paper_bgcolor':'white', 'plot_bgcolor' : "white",
    'yaxis': {'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black", 
              'title': {'text': "3m/3m Rate "}
             }, 
    'title': {'text': "US Real GDP Time Series Predictions<br>Covering All Available Historical Data"}
}
    )
all_vis = []
q1_vis = []
q2_vis = []
q3_vis = []
q4_vis = []
for plt in fig.data:
    splits = plt.name.split(' - ') 
    if len(splits)== 2:
        all_vis.append(False)
        if "Q1" in splits[1]:
            q1_vis.append(True)
        else:
            q1_vis.append(False)
            
        if "Q2" in splits[1]:
            q2_vis.append(True)
        else:
            q2_vis.append(False)
        
        if "Q3" in splits[1]:
            q3_vis.append(True)
        else:
            q3_vis.append(False)
            
        if "Q4" in splits[1]:
            q4_vis.append(True)
        else:
            q4_vis.append(False)
    else:
        all_vis.append(True)
        q1_vis.append(False)
        q2_vis.append(False)
        q3_vis.append(False)
        q4_vis.append(False)

fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            active=0,
            x=0.57,
            y=1.2,
            buttons=list([
                dict(label="Full",
                     method="update",
                     args=[{},{"xaxis": 
                                  {'range': [yld_curve.index.min(), yld_curve.index.max()] 
                                   }, 
                            "title": "US Real GDP Time Series Predictions<br>Covering All Available Historical Data",
                            "yaxis": {'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"}
                            }, 
                           [0]]),
                dict(label="80s",
                     method="update",
                     args=[{},{"xaxis": 
                                  {'range': [pd.Timestamp('1985-01-01'), pd.Timestamp('1990-01-01')] 
                                   }, 
                            "title": "Black Monday Crash to Black Friday Crash",
                            "yaxis": {'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"}
                            }, 
                           [0]]),
                dict(label="Dotcom",
                     method="update",
                     args=[{},{"xaxis": 
                                  {'range': [pd.Timestamp('1995-01-01'), pd.Timestamp('2000-04-01')], 
                                   }, 
                            "title": "DotCom Crash to Lehman Default",
                            "yaxis": {'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"}
                            }, 
                           [0]]),
                dict(label="Pre Housing",
                     method="update",
                     args=[{},{"xaxis": 
                                  {'range': [pd.Timestamp('2000-04-01'), pd.Timestamp('2008-10-01')], 
                                   }, 
                            "title": "DotCom Crash to Lehman Default",
                            "yaxis": {'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"}
                            }, 
                            
                           [0]]),
                dict(label="Pre 2016",
                     method="update",
                     args=[{}, {"xaxis": {'range': [pd.Timestamp('2008-10-01'), pd.Timestamp('2016-10-01')],
                                        },
                            "title": "Lehman Default to Brexit & Trump Election Beginning",
                            "yaxis": {'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"}
                            }, 
                           [0]]),
                dict(label="Present",
                     method="update",
                     args=[{},{"xaxis": {'range': [pd.Timestamp('2016-10-01'), pd.Timestamp('2022-12-31')],
                                        },
                            "title": "Post Brexit & Trump Election to Present", 
                            "yaxis": {'gridcolor': 'lightgrey', 'zeroline': True, 'zerolinecolor': "black"} 
                            },
                            
                           [0]]),

            ]),
        ),
        dict(
            type="buttons",
            buttons=[
                dict(label="All",
                     method="update",
                     args=[{
                         'visible': all_vis
                     }, {}]),
                dict(label="Q1 Only",
                     method="update",
                     args=[{
                         'visible': q1_vis
                         
                     },{}]),
                dict(label="Q2 Only",
                     method="update",
                     args=[{
                         'visible': q2_vis
                         
                     },{}]),
                dict(label="Q3 Only",
                     method="update",
                     args=[{
                         'visible': q3_vis
                         
                     },{}]),
                dict(label="Q4 Only",
                     method="update",
                     args=[{
                         'visible': q4_vis
                         
                     },{}]),
                
            ],
        )
    ]
)
        
   

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, d2_tweedie_score

In [ ]:
subset = gdp_new[['Real_GDP_QoQ', 'predict1', 'predict2', 'predict3']].dropna()
print("AR12 error: ", f"{mean_squared_error(subset['Real_GDP_QoQ'], subset['predict1']) :.2f}","Tweedie: ", f"{d2_tweedie_score(subset['Real_GDP_QoQ'], subset['predict1'])*100 :.2f}%", "R2: ", f"{r2_score(subset['Real_GDP_QoQ'], subset['predict1'])*100 :.2f}%")
print("AR8 error: ", f"{mean_squared_error(subset['Real_GDP_QoQ'], subset['predict2']) :.2f}", "Tweedie: ",  f"{d2_tweedie_score(subset['Real_GDP_QoQ'], subset['predict2'])*100 :.2f}%","R2: ", f"{r2_score(subset['Real_GDP_QoQ'], subset['predict2'])*100 :.2f}%")
print("AR1 error: ", f"{mean_squared_error(subset['Real_GDP_QoQ'], subset['predict3']) :.2f}", "Tweedie: ",  f"{d2_tweedie_score(subset['Real_GDP_QoQ'], subset['predict3'])*100 :.2f}%","R2: ", f"{r2_score(subset['Real_GDP_QoQ'], subset['predict3'])*100 :.2f}%")

## Observations from the first predictions

The AR(1) was remarkably close in its Q3 prediction for this month as was the Q3 quarter prediction using an AR(12) but only focusing on the quarters time series.

Again, I will never say that Time Series Forecasting is a reliable predictor - especially given the Tweedie and R2 score basically suggest that we should do the opposite what the model says (though with AR1 model -less variance) - that being said, I still think that they can still useful indicators down the line when applying more sophisticated ML models (GB regression trees, Random Forest using Retail Trade, jobless claims, etc). 

Down the line we will need to use Granger causality tests to see whether different series of data will have the most correlation to GDP from a lagged capacity.

<a id="money_supply"></a>
# Money Supply

[Return Home](#tbc)

In [ ]:
money_supply = web.DataReader(["BOGMBASE", "M1SL", "M2SL"], 'fred', datetime(1970, 6, 1), datetime.now())

In [ ]:
fig = make_subplots(rows=2, cols=2, shared_xaxes=True, subplot_titles=("M1 & M2 Money Supply", "M0 Money Supply", "M1 & M2 MoM Changes", "M0 MoM Changes"), vertical_spacing=.07, horizontal_spacing=.08)
fig.add_trace(go.Bar(x=money_supply.index, y=money_supply['M2SL'].values, name="M2"), row=1, col= 1)
fig.add_trace(go.Bar(x=money_supply.index, y=money_supply['M1SL'].values, name="M1"), row= 1, col= 1)
fig.add_trace(go.Bar(x=money_supply.index, y=money_supply['BOGMBASE'].values, name="M0"),row=1, col=2)
fig.add_trace(go.Scatter(x=money_supply.index, y=money_supply['M2SL'].pct_change().values, name="M2 pct change"), 2, 1)
fig.add_trace(go.Scatter(x=money_supply.index, y=money_supply['M1SL'].pct_change().values, name="M1 pct change"), 2, 1)
fig.add_trace(go.Scatter(x=money_supply.index, y=money_supply['BOGMBASE'].pct_change().values, name="M0 pct change"),row=2, col=2)
fig.update_layout(barmode='stack', paper_bgcolor='white', plot_bgcolor = "white", title={'text': "Money Supply"}, yaxis3={'tickformat': "0%"}, yaxis4={'tickformat': "0%"},
height=800, xaxis={
    'range': [pd.Timestamp('1990-01-01'), pd.Timestamp('2023-01-01')]}, xaxis2={'range': [pd.Timestamp('1990-01-01'), pd.Timestamp('2023-01-01')]
    }
    )




## Observations

The Money supply has had generally consistent growth (with May 2020 being an outlier month in M1). While monetary base has grown steadily, with risk of severe recession on the horizon there is a chance of mild contraction in the M0. Unfortunately no data for the 1980s period, would have been interesting to see what was the impact after the crash of 1987.

<a id='fx'></a>

In [ ]:
developed_fx = web.DataReader(["EURUSD=X", "USDSEK=X", "GBPUSD=X", "USDJPY=X", "USDDKK=X", "USDNOK=X"], 'yahoo', datetime(1990, 6, 1), datetime.now())



In [ ]:
em_fx = web.DataReader(["TRY=X", "ZAR=X", "RUB=X","BRL=X", "CZK=X", "PLN=X", "MXN=X", "USDKRW=X", "USDCNY=X", "USDCNH=X", "HUF=X", "COP=X", "CLP=X"], 'yahoo', datetime(1990, 6, 1), datetime.now())

In [ ]:
data = []
for fx in ["EURUSD=X", "USDSEK=X", "GBPUSD=X", "USDJPY=X", "USDDKK=X", "USDNOK=X"]:
    sub = developed_fx.xs(fx, axis=1, level=1)
    sub = sub.dropna()
    fx_name = fx.split("=")[0]
    data.append(go.Candlestick(x=sub.index,
                open=sub['Open'], high=sub['High'],
                low=sub['Low'], close=sub['Close'], name=fx_name))    

In [ ]:
fig = go.Figure(data)

fig.update_layout(xaxis_rangeslider_visible=False, title={'text': "FX Historical"})

In [ ]:
em_fx.loc[:, ('Open')]

<a id="granger_causality_inflation"></a>
# Granger Causality tests on spot inflation vs inflation expecations

(the results should be obvious, but let's see anyways)

<i> Enhancement on this section is currently being developed</i>

In [ ]:
granger_1y = sta.grangercausalitytests(pd.concat([inflation_curves, spot_inflation], axis=1).dropna()[['CORESTICKM158SFRBATL', 'EXPINF1YR']], 6)

In [ ]:
granger_2y = sta.grangercausalitytests(pd.concat([inflation_curves, spot_inflation], axis=1).dropna()[['CORESTICKM158SFRBATL', 'EXPINF2YR']], 6)

In [ ]:
granger_10y = sta.grangercausalitytests(pd.concat([inflation_curves, spot_inflation], axis=1).dropna()[['CORESTICKM158SFRBATL', 'EXPINF10YR']], 6)

# Observations
A rough granger causality test yields that spot inflation observations have a high influence across the curve lagged 1-2 month.